# AssurPrime : Saurez-vous prédire la prime d'assurance ?

<p align="center">
  <img src="https://challengedata.ens.fr/logo/public/CA_assurances_RVB_sans-raison-d%C3%AAtre_%C3%A9v%C3%A9nementiel_Mg5WVL6.png" width="350" title="Crédit Agricole Assurances">
  <p align="center">
    <a href="https://colab.research.google.com/github/auduvignac/challengedata_ens_AssurPrime/blob/main/notebooks/exploration/assurprim_modeling_workflow.ipynb" target="_blank">
      <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Ouvrir dans Google Colab"/>
    </a>
  </p>
</p>

# Présentation complète du projet

## Contexte

Crédit Agricole Assurances est une filiale du Groupe Crédit Agricole dédiée à l’assurance, faisant de celui-ci un acteur multi-expert de la bancassurance et le 1er bancassureur en Europe.
Crédit Agricole Assurances regroupe plusieurs entités, dont Predica et Pacifica, qui proposent une large gamme d’assurances aux particuliers, aux exploitants agricoles, aux professionnels et aux entreprises. Crédit Agricole Assurances s’engage à offrir des solutions innovantes et adaptées aux besoins des clients, tout en favorisant le développement durable et la responsabilité sociale.
Au sein de l’Académie Data Science du groupe, l’objectif est de participer activement à la montée en compétences des collaborateurs, de partager des connaissances et d’identifier de nouveaux usages.

## Objectif

Le contrat Multirisque Agricole, géré par Pacifica, est souscrit par les agriculteurs pour sécuriser leur exploitation. Il couvre l’activité professionnelle, les dommages aux bâtiments d’exploitation, le matériel stocké, ainsi que la protection financière et juridique. Ce contrat garantit à l’assuré une couverture efficace et durable, assurant ainsi la continuité de son activité en cas de sinistre, tant sur le plan matériel que financier.

Actuellement, le risque d’incendie constitue une part majeure de la charge sinistre du contrat Multirisque Agricole, ce qui en fait un enjeu clef à modéliser avec précision.

L’objectif est d’identifier le meilleur modèle pour prédire la prime pure incendie, en utilisant :
- Un modèle pour la Fréquence,
- Un modèle pour le Coût moyen.

La variable cible finale, la charge, est obtenue en multipliant la fréquence, le coût moyen, et le nombre d’années depuis la souscription du contrat (la variable `ANNEE_ASSURANCE`).


## Description des données

Un fichier supplémentaire est mis à disposition regroupant toutes les variables disponibles, accompagnées de leur description. Ce fichier inclut :

- **Les variables cibles** : `FREQ`, `CM`, et `CHARGE`
- **Données géographiques** : département, données météorologiques, etc.
- **Données spécifiques au contrat**, notamment :
    - L’activité de l’assuré (cultivateur, polyculteur, etc.)
    - Les indicateurs de souscription des garanties
    - Le nombre de bâtiments, de salariés, et de sinistres déclarés lors de la souscription
    - **Données de surface** : surfaces des bâtiments (élevage, exploitation, etc.), anonymisées en `surface1`, `surface2`, etc., pour garantir la confidentialité
    - **Données de capitaux** : capitaux assurés pour différentes options (vol, serres, etc.), anonymisés en `capital1`, `capital2`, etc.
    - **Données liées à la prévention** : présence d’équipements (extincteurs, structure en bois, etc.), anonymisées en `prev1`, `prev2`, etc.


## Description du *benchmark*

### Objectif du *challenge*

L’objectif de ce challenge est de comparer les performances des modèles développés dans le cadre de cette compétition avec celles d’un modèle de référence basé sur des **GLM (Generalized Linear Models)** classiques.

### Structure du benchmark

Le benchmark repose sur deux modèles **GLM** distincts :

- Fréquence des sinistres :
    - Distribution : *Loi de Poisson* ;
    - Fonction de lien : *Log*.

- Coût moyen d’un sinistre :
    - Distribution : *Tweedie* ;
    - Fonction de lien : *Log*.

### Évaluation

L’évaluation des modèles repose sur une métrique unique : **RMSE (Root Mean Square Error)**, définie par la formule suivante :

$$\text{RMSE} = \sqrt{\frac{1}{n} \sum\limits_{i=1}^{n} (y_i - \hat{y}_i)^2}$$

où :
- $y_i$ représente la valeur réelle ;
- $\hat{y}_i$ représente la valeur prédite ;
- $n$ est le nombre d’observations.

L’idée reste d’évaluer dans quelle mesure les approches proposées permettent de dépasser les performances des modèles standards tout en prenant en compte :

- La précision des prédictions ;
- Les aspects d’interprétabilité et d’efficacité ;
- Les contraintes métier associées.

# Résumé

Le but est de construire un pipeline de modélisation permettant d’estimer la **charge sinistre incendie**, via deux composantes principales :

- La **fréquence des sinistres** (`FREQ`)
- Le **coût moyen des sinistres** (`CM`)

La variable cible finale est définie comme : `CHARGE = FREQ × CM × ANNEE_ASSURANCE`.

## Données disponibles

- **Cibles** :
  - `FREQ`;
  - `CM`;
  - `CHARGE`.
- **Informations géographiques** :
  - département;
  - météo.
- **Descripteurs du contrat** :
  - activité assurée;
  - garanties souscrites;
  - nombre de bâtiments;
  - surface;
  - capitaux;
  - équipements de prévention.
- **Variables anonymisées** : certaines variables sensibles ont été anonymisées pour des raisons de confidentialité. Bien que leur signification exacte soit masquée, leur typologie (quantitative, binaire, etc.) permet d’en exploiter le contenu via des méthodes d’analyse statistique et de modélisation. Elles se présentent sous forme de groupes de variables thématiques telles que :
  - `surface1`, `surface2`, ... : indicateurs de **surfaces agricoles** (bâtiments d’élevage, stockage, exploitation, etc.)
  - `capital1`, `capital2`, ... : **capitaux assurés** pour différentes garanties optionnelles (ex. vol, serres, machines)
  - `prev1`, `prev2`, ... : **mesures de prévention** contre les incendies ou sinistres (présence d’extincteurs, structures en matériaux spécifiques, etc.)


## Modèle de référence (benchmark)

Un double modèle **GLM** sert de base comparative :

| Modèle        | Distribution | Fonction de lien |
|---------------|--------------|------------------|
| Fréquence     | Poisson      | Log              |
| Coût moyen    | Tweedie      | Log              |

Ces modèles sont évalués selon la **Root Mean Square Error (RMSE)** :$\sqrt{\frac{1}{n} \sum\limits_{i=1}^{n} (y_i - \hat{y}_i)^2}$, sur la variable finale `CHARGE`.

## Contenu du notebook

Ce notebook comprend les étapes suivantes :

1. **Exploration des données**
2. **Préparation des variables** (feature engineering, nettoyage, encodage)
3. **Modélisation fréquence & coût moyen**
4. **Évaluation des performances**
5. **Analyse des résidus et interprétation**

# Bibliothèques utilisées

Cette section regroupe l’installation et l’importation des bibliothèques nécessaires pour l’ensemble du pipeline : de l’analyse exploratoire à la modélisation, en passant par le prétraitement des données et l’évaluation des performances.

### Manipulation de données
- `pandas`, `numpy` : gestion et transformation de données tabulaires et numériques.

### Visualisation
- `matplotlib.pyplot`, `seaborn` : création de graphiques pour l’exploration visuelle des données et l’interprétation des résultats.

### Statistiques et tests
- `scipy.stats.chi2_contingency` : test d’indépendance du khi² sur des variables qualitatives.

### Machine Learning
- `scikit-learn` : 
  - Prétraitement : `SimpleImputer`, `StandardScaler`
  - Modélisation : régression logistique, Ridge, Random Forest, Gradient Boosting
  - Validation croisée, tuning et métriques : RMSE, F1-score, AUC, etc.
- `xgboost` : algorithmes de boosting performants via `XGBClassifier` et `XGBRegressor`.

### Encodage des variables catégorielles
- `category_encoders.CountEncoder` : encodage des modalités par fréquence d’apparition.

### Affichage dans le notebook
- `IPython.display.display` : affichage ciblé et lisible des DataFrames ou objets complexes.


In [ ]:
!pip install -q category-encoders \
                matplotlib \
                numpy \
                pandas \
                scikit-learn \
                seaborn \
                xgboost

In [ ]:
import re

# Standard library (ex. pathlib, math, os, sys)
from pathlib import Path

# Third-party libraries (ex. numpy, pandas, matplotlib, scikit-learn)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from category_encoders import CountEncoder
from IPython.display import HTML, display
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import (
    mean_squared_error,
    roc_auc_score,
)
from sklearn.model_selection import (
    train_test_split,
)

# Chargement et aperçu des données

Chargement des fichiers de données d'entraînement à partir des sources distantes. Les fichiers comprennent :

- `X_train` : variables explicatives issues des contrats d’assurance ;
- `y_train` : variables cibles associées :
  - `FREQ` : fréquence des sinistres incendie ;
  - `CM` : coût moyen par sinistre ;
  - `CHARGE` : charge totale sinistre incendie (`CHARGE = FREQ × CM × ANNEE_ASSURANCE`).

Le chargement des fichiers repose sur une stratégie de *fallback* : les données sont d’abord recherchées en local (chemins absolus), et en cas d’absence, elles sont automatiquement téléchargées depuis des URLs distantes, puis conservées localement pour les exécutions futures.

In [ ]:
# Définition des chemins locaux
local_x_path = Path("../../data/raw/x_train.csv").resolve()
local_y_path = Path("../../data/raw/y_train.csv").resolve()

# URLs distantes (GitHub)
X_train_url = (
    "https://media.githubusercontent.com/media/auduvignac/"
    "challengedata_ens_AssurPrime/refs/heads/main/data/raw/x_train.csv"
)
y_train_url = (
    "https://media.githubusercontent.com/media/auduvignac/"
    "challengedata_ens_AssurPrime/refs/heads/main/data/raw/y_train.csv"
)


def load_dataset(local_path, url, name):
    """
    Charge un fichier CSV depuis un chemin local, ou le télécharge depuis
    une URL si le fichier n'existe pas localement.

    Le fichier est ensuite sauvegardé localement pour les exécutions
    futures (fallback).

    Parameters
    ----------
    local_path : pathlib.Path
        Chemin vers le fichier local.
    url : str
        URL distante du fichier CSV.
    name : str
        Nom affiché lors du chargement (utilisé pour les logs).

    Returns
    -------
    pandas.DataFrame
        Le contenu du fichier CSV sous forme de DataFrame.
    """
    if local_path.exists():
        print(f"{name} chargé depuis : {local_path}")
        return pd.read_csv(local_path)
    else:
        print(f"{name} non trouvé localement, téléchargement depuis l'URL...")
        df = pd.read_csv(url)
        local_path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(local_path, index=False)
        print(
            f"{name} téléchargé et enregistré localement dans : {local_path}"
        )
        return df


# Chargement des jeux de données
try:
    print("Chargement des données...")
    X_train = load_dataset(local_x_path, X_train_url, "X_train")
    y_train = load_dataset(local_y_path, y_train_url, "y_train")
    print("Données chargées avec succès.\n")

except Exception as e:
    print("Échec du chargement des données.")
    print("Vérifiez la validité des URL ou la connexion internet.")
    print(e)

Une fois le chargement des données effectué, il convient tout d’abord de vérifier que `X_train` et `y_train` possèdent le même nombre de lignes. Un aperçu des premières lignes du jeu d’entraînement est ensuite présenté afin d’en examiner la structure et le contenu.

In [ ]:
print(f"Dimensions de X_train : {X_train.shape}")
print(f"Dimensions de y_train : {y_train.shape}")

if len(X_train) == len(y_train):
    print("Les jeux de données disposent bien du même nombre d'observations.")
else:
    print(
        "Les jeux de données ne disposent pas du même nombre d'observations."
    )

La correspondance en nombre d’observations entre `X_train` et `y_train` étant assurée, nous poursuivons avec une inspection des premières colonnes de `X_train`, afin d’identifier la nature et l’hétérogénéité des variables disponibles.

## Aperçu des variables explicatives (`X_train`)

Le jeu de données `X_train` contient les **variables explicatives** issues des contrats d’assurance souscrits dans le cadre du produit Multirisque Agricole.  
Chaque ligne correspond à **un contrat**, et chaque colonne décrit une caractéristique associée.

Ci-dessous, les premières lignes de `X_train` sont affichées pour donner un aperçu de la structure des données. disponibles.

In [ ]:
# aperçu de X_train
print("Aperçu de X_train :")
display(X_train.head())

Les premières colonnes mettent en évidence la richesse du jeu de données :

- **Variables catégorielles** liées au contrat : `ACTIVIT2`, `VOCATION`, `TYPERS`, etc.
- **Caractéristiques techniques ou déclaratives** : `CARACT1`, `CARACT2`, etc.
- **Données météorologiques et historiques** : variables commençant par `NBJRR`, `RR_VOR`, `RRAB_VOR`, etc.
- **Variables numériques continues** comme `ANNEE_ASSURANCE`, indiquant l’ancienneté du contrat.

Certaines colonnes contiennent :

- Des **valeurs manquantes** qu’il faudra prendre en compte dans le cadre du prétraitement ;
- Des données **anonymisées ou encodées** (`surface1`, `capital1`, `prev1`, etc.) pour des raisons de confidentialité.

### Statistiques descriptives de `X_train`

Examinons les statistiques descriptives de `X_train` pour mieux comprendre la distribution des variables, identifier les valeurs manquantes et détecter les éventuelles anomalies.

Pour cela, une méthode `scrollable_describe` a été implémentée. Elle s’appuie sur la méthode `describe()` de `pandas` pour générer un résumé statistique des colonnes numériques du DataFrame, tout en encapsulant l’affichage dans un bloc défilant afin de mutualiser les appels et et assurer un lisibilité plus aisée.

In [ ]:
def scrollable_describe(df, height=400, round=2):
    """
    Affiche une version transposée et scrollable du résumé statistique
    d'un DataFrame (résultat de .describe()).

    Parameters
    ----------
    df : pd.DataFrame
        jeu de données à décrire
    height : int
        hauteur de la boîte scrollable en pixels
    round : int
        nombre de décimales pour l'arrondi des valeurs
    Returns
    -------
    HTML : IPython.display.HTML
        une représentation HTML scrollable du résumé statistique
    """
    desc = df.describe().T.round(round)
    html = desc.to_html()
    return HTML(
        f"""
        <div style="height:{height}px; overflow:auto;
                    border:1px solid lightgray; padding:10px">
            {html}
        </div>
        """
    )

Le tableau ci-dessous fournit un résumé statistique des **94 variables numériques** de `X_train`, calculé sur les colonnes de types `int64` et `float64`.

Ce résumé permet notamment de :
- Identifier les **ordres de grandeur** et la **variabilité** des variables ;
- Détecter la présence de **valeurs manquantes** (via les valeurs de `count`) ;
- Repérer les **variables constantes** ou à faible dispersion. Pour cela, l’écart-type (*standard deviation*), permettant de mesurer à quel point les valeurs d’une variable sont dispersées autour de la moyenne, sera particulièrement utile. Plus l'écart-type est élevé, plus la variabilité est élevée (i.e. les valeurs sont dispersées), ce qui peut laisser supposer que la variable apporte une information discriminante pour la modélisation. À l’inverse, un écart-type proche de 0 indique une faible variabilité, ce qui peut suggérer que la variable n’apporte pas d’information discriminante pour la modélisation ;
- Observer les distributions à travers les **quartiles** (`25%`, `50%`, `75%`) ;
- Identifier d’éventuelles **valeurs extrêmes** (`min`, `max`) pouvant signaler des outliers.

L’analyse de ce tableau permettra de prioriser les traitements à venir (imputation, transformation, normalisation, etc.).

In [ ]:
scrollable_describe(X_train)

### Premiers constats quant aux statistiques descriptives relatives à `X_train`

- Plusieurs variables comme `CARACT2`, `RISK5` ou `ZONE_VENT` affichent un nombre d’observations inférieur au nombres de lignes associés au *dataset* (`count = 383 610`), indiquant la présence de **valeurs manquantes** (déjà constatée dans l’aperçu des données) ;
- Certaines variables ont des **valeurs maximales très élevées** (`CA1`, `CA2` jusqu’à `30 000`, `CA3` jusqu’à `50 000`), ce qui pourrait **influencer fortement la distribution**, notamment en cas de forte asymétrie ou de valeurs extrêmes isolées (*outliers*) ;
- Des colonnes comme `EQUIPEMENT1` ou `EQUIPEMENT3` semblent **binaires ou quasi-binaires** (`min = 0`, `max = 1`, `mean` faible), ce qui suggère qu'elles peuvent être traitées comme des **indicateurs logiques** ;
- La colonne `ANNEE_ASSURANCE` varie entre environ `0.0027` et `2.0`, ce qui suggère qu’il s’agit d’un **ratio (ou prorata) d’années** plutôt qu’un entier représentant un nombre d’années complètes ;
- Certaines variables telles que `DEROG15`, `TYPBAT2` ou `CARACT5` présentent une **variabilité très faible** (majorité de modalités identiques), ce qui peut **limiter leur pouvoir discriminant** en modélisation et justifie une réflexion sur leur maintien ;
- D'autres variables comme `KAPITAL10`, `KAPITAL11` ou `KAPITAL12` montrent une grande **hétérogénéité en valeur**, parfois avec une majorité de zéros, ce qui pourrait indiquer une **sparsité partielle** qu'il conviendra d’examiner plus précisément ;
- Enfin, certaines familles de variables (`SURFACE`, `NBBAT`, `KAPITAL`, `RISK`) présentent des structures répétées (suffixes numériques), suggérant un **modèle de codage hiérarchique ou multi-dimensionnel**, qu’il pourrait être utile de regrouper ou résumer par des agrégats statistiques lors du prétraitement.

Cette analyse guidera la **phase de nettoyage** et de **sélection de variables**, en complément de la visualisation et du traitement des valeurs manquantes.

Dans une démarche analogue, examinons à présent la structure de `y_train`, qui regroupe les **variables cibles** indispensables à la modélisation.

## Aperçu des variables cibles (`y_train`)

Le jeu de données `y_train` contient les **variables cibles** associées à chaque contrat :

- `FREQ` : fréquence des sinistres liés à l’incendie (éventuellement nulle) ;
- `CM` : coût moyen des sinistres (en euros) ;
- `ANNEE_ASSURANCE` : nombre d’années depuis la souscription ;
- `CHARGE` : charge sinistre totale (`CHARGE = FREQ × CM × ANNEE_ASSURANCE`)

Un extrait de `y_train` est présenté ci-dessous permettant ainsi d'en observer la structure.

In [ ]:
# aperçu de y_train
print("Aperçu de y_train :")
display(y_train.head())

Un aperçu des premières lignes de `y_train` permet de visualiser la structure du jeu de données cible. Les variables `FREQ`, `CM` et `CHARGE` y sont systématiquement nulles pour les premières observations, ce qui reflète des cas sans sinistre déclaré. Ce constat préliminaire met en évidence un déséquilibre important dans la variable `CHARGE`, à prendre en compte lors de la modélisation. La variable `ANNEE_ASSURANCE`, exprimée sous forme décimale, suggère une granularité infra-annuelle.

### Elements de précisions sur la variable `ANNEE_ASSURANCE`

La variable `ANNEE_ASSURANCE` semble représenter le **temps écoulé depuis la souscription du contrat**, exprimé en **fraction d’année**. Par exemple, une valeur de `0.5` correspondrait à environ six mois d’ancienneté, tandis qu’une valeur de `1.0` indiquerait une année complète.

Plus formellement, cette variable pourrait être interprétée comme un **rapport entre le nombre de jours écoulés depuis la souscription et le nombre de jours dans une année**, selon la formule suivante :

$$
\text{ANNEE\_ASSURANCE} = \frac{\text{Nombre de jours depuis la souscription}}{365}
$$

Cette granularité permet :
- de modéliser plus finement les effets du temps d’exposition au risque dans un cadre assurantiel ;
- d’éviter les pertes d’information liées à un arrondi à l’unité (par exemple en années entières).

Dans les modèles assurantiels, on modélise souvent la durée d’exposition au risque sous forme fractionnaire (i.e. en années décimales), notamment pour :
- Modéliser les sinistres en fréquence :
La fréquence des sinistres est souvent rapportée à une année. Si un contrat n’est actif que 6 mois, il est logique d’ajuster son poids dans le calcul :
$$
\text{Fréquence annualisée} = \frac{\text{Nombre de sinistres}}{\text{Durée d'exposition en années}}
$$
- Pondérer les charges sinistres :
Dans une modélisation de charge sinistre, une observation ayant été exposée moins longtemps au risque doit logiquement avoir une influence proportionnelle à cette durée.
- Utiliser des modèles adaptés : Poisson, Gamma ou encore Tweedie

Dans la continuité de l’analyse effectuée sur `X_train`, étoffons le propos précédent avec les statistiques descriptives de `y_train`.

### Statistiques descriptives de `y_train`

In [ ]:
# le nombre de variables présentes dans y_train étant plus faible que dans
# X_train, on peut afficher le résumé statistique sans scroll et ainsi
# affecter à height une valeur plus petite, en l'occurrence 160 pixels.
scrollable_describe(y_train, height=160)

### Premiers constats quant aux statistiques descriptives relatives à `y_train`

L’analyse statistique des variables cibles met en évidence plusieurs éléments majeurs à prendre en compte dans la suite du traitement :

- La variable `FREQ`, représentant la fréquence de sinistres, présente une valeur moyenne extrêmement faible (`0.01`), ce qui reflète un déséquilibre important entre les contrats avec et sans sinistre. Une valeur maximale de `182.5` est observée, suggérant un cas atypique ou potentiellement erroné qui mérite une vérification approfondie.

- La variable `CM` (coût moyen des sinistres), présente une distribution très étendue, avec une moyenne de `182.52`, un écart-type très élevé ($\sigma = 6 699.97$), et des valeurs aberrantes notables allant jusqu’à `500 000`. La présence d'une valeur minimale négative (`-5 751.0`) est incohérente et indique probablement une erreur d'encodage ou un cas particulier.

Les cas de figures pouvant exprimer un `CM` négatif sont les suivants :

- **Recours ou subrogation** : remboursement par un tiers responsable d’un sinistre ;
- **Correction comptable** ou **ajustement rétroactif** d’un sinistre ;
- **Remboursement net** en faveur de l’assureur dans des cas exceptionnels ;
- **Erreur d’encodage** ou convention spécifique de saisie.

Ces explications demeurent spéculatives. Une clarification avec les équipes métiers ou une exploration plus approfondie du contexte des données serait nécessaire pour statuer sur la légitimité ou non de ces valeurs. Malheurseusement dans le cadre du projet aucune information complémentaire n'est fournie.

Selon l’impact observé, une décision de nettoyage ou de transformation des données pourrait être envisagée lors du prétraitement.

- La variable `CHARGE`, calculée comme le produit `FREQ × CM × ANNEE_ASSURANCE`, hérite logiquement de cette forte hétérogénéité. Elle présente une moyenne de `186.09`, mais également une grande dispersion ($ \sigma \approx 6 800$) et une valeur maximale de `552 000`, soulignant la nécessité d’un traitement robuste des valeurs extrêmes. Une valeur minimale négative est également observée, ce qui renforce l’idée de valeurs anormales dans `CM`.

- Enfin, la variable `ANNEE_ASSURANCE` varie de `0.0` à `2.0`, ce qui indique vraisemblablement une représentation continue d’un ratio d’ancienneté ou d’année d’assurance, et non une valeur entière comme une simple année calendaire.

Ces constats mettent en évidence la nécessité de traitements spécifiques lors de la phase de prétraitement, en particulier sur les valeurs extrêmes et aberrantes, ainsi que sur la rareté structurelle des sinistres, qui devra être prise en compte dans le choix des algorithmes, des métriques d’évaluation et des méthodes de validation.

Pour étoffer l'analyse, vérifions dans quelle mesure ce déséquilibre est présent dans l’ensemble du jeu de données, en calculant le **nombre et le pourcentage de lignes** avec `FREQ = 0`.

In [ ]:
# Calcul du nombre et du pourcentage de lignes avec FREQ nulle
nb_freq_zero = (y_train["FREQ"] == 0).sum()
pourcentage_freq_zero = nb_freq_zero / len(y_train) * 100

print(f"Nombre de lignes avec FREQ = 0 : {nb_freq_zero}")
print(f"Nombre de lignes totales : {len(y_train)}")
print(f"Proportion de FREQ = 0 : {pourcentage_freq_zero:.2f}%")

Ce résultat met en évidence les éléments suivants :

- Une **écrasante majorité** des contrats présentent une fréquence de sinistres nulle :
  **380 716 lignes sur 383 610**, soit **99,25 %** des observations, ont `FREQ = 0`.  
  Cela signifie que la variable `CHARGE` est également nulle dans la quasi-totalité des cas ;
  
- Ce **déséquilibre extrême**, typique des données assurantielles, où les sinistres sont rares mais coûteux, aura un impact important sur :
  - Le choix des modèles (ex. classification binaire et régression) ;
  - Les métriques d’évaluation (ex. RMSE pondérée, F1-score sur sinistrés) ;
  - Les stratégies de sous-échantillonnage, sur-échantillonnage ou filtrage.

## Synthèse générale de l’analyse exploratoire

L’inspection de `X_train` et `y_train` a permis de dégager plusieurs éléments clefs qui orienteront les étapes de prétraitement et de modélisation :

#### Concernant les variables explicatives (`X_train`)
- Le jeu de données est riche et hétérogène, incluant des **variables catégorielles**, **numériques**, **techniques**, ainsi que des **données météorologiques et historiques** ;
- De nombreuses **valeurs manquantes** sont présentes dans certaines colonnes, impliquant des stratégies d’imputation ciblées ;
- Plusieurs variables présentent des **valeurs extrêmes** ou une **forte dispersion**, qui pourraient biaiser certains algorithmes sensibles à l’échelle des données (ex. régression linéaire, KNN) ;
- Certaines colonnes montrent une **faible variabilité**, voire sont quasi constantes, posant la question de leur **pertinence prédictive** ;
- Des motifs récurrents dans les noms de colonnes (ex. `KAPITAL`, `SURFACE`, `NBBAT`) laissent entrevoir des structures hiérarchiques ou redondantes, propices à une agrégation ou à une sélection ;
- La variable `ANNEE_ASSURANCE` est exprimée en **fraction d’année**, élément pertinent pour modéliser l’exposition au risque mais qui nécessite une attention particulière lors de la modélisation.

#### Concernant les variables cibles (`y_train`)
- Le champ `CHARGE`, qui constitue la cible principale, est dérivé du produit `FREQ × CM × ANNEE_ASSURANCE`, ce qui en fait une **variable composite** ;
- Un **déséquilibre extrême** est observé : plus de **99 %** des contrats présentent `FREQ = 0`, conduisant mécaniquement à `CHARGE = 0`. Cette rareté des sinistres est un défi classique en assurance, à traiter via des approches robustes ;
- La variable `CM` (coût moyen des sinistres) présente des **valeurs anormales**, notamment des valeurs **négatives** ou **très élevées**, qui devront faire l’objet d’un traitement particulier (nettoyage ou transformation) ;
- `ANNEE_ASSURANCE` semble être une **valeur continue** (et non entière), représentant un **ratio temporel** ou une part d’année d’exposition au risque.

Ces constats initiaux soulignent la nécessité d’une analyse exploratoire approfondie, afin de mieux cerner la structure, la qualité et la distribution des données. Cette inspection permettra d’orienter les décisions à venir, notamment en matière de prétraitement rigoureux : gestion des valeurs manquantes, détection des anomalies, et adaptation des variables aux exigences des modèles prédictifs.

# Analyse exploratoire approfondie des données et nettoyage progressif

L’analyse exploratoire précédente a permis de mettre en évidence plusieurs caractéristiques structurelles du jeu de données.
Pour compléter cette exploration, un approfondissement du jeu de données associé à un nettoyage progressif de `X_train` et `y_train` est entrepris.
Cette démarche vise à affiner la compréhension des données, à identifier les variables pertinentes et à préparer le terrain pour les étapes de modélisation ultérieures.

Le tableau ci-dessous résume les principales étapes de nettoyage et de préparation des données, ainsi que leurs objectifs respectifs :


| Étape                                             | Objectif                                                           |
| ---------------------------------------------     | -------------------------------------------------------            |
| Répartition des types de variables dans `X_train` | identifier la nature des variables présentes dans le jeu `X_train` |
| Suppression des variables non informatives        | Cardinalité = 1, trop de valeurs manquantes                        |
| Harmonisation des types                    | Forcer les `int`, `float`, `category`, `bool`, etc.     |
| Standardisation des modalités              | Supprimer les blancs, corriger les libellés incohérents |
| Détection + marquage des valeurs atypiques | Par seuils, IQR ou z-score                              |
| Réduction des modalités rares              | Grouper les catégories peu fréquentes                   |
| Traitement des valeurs manquantes           | Imputation, remplacement ou exclusion raisonnée         |

Préalablement, une fonction `scrollable_describe` est définie pour afficher les statistiques descriptives de manière défilante, facilitant ainsi la visualisation des résultats sur l’ensemble des variables.
De plus, une copie de `X_train`, intitulée `X_train_clean`, est créée pour préserver l’intégrité des données originales tout au long du processus de nettoyage.

In [ ]:
def display_scrollable_df(
    df, height=300, border_color="lightgray", padding=10, index=False
):
    """
    Affiche un DataFrame dans une boîte HTML scrollable.

    Paramètres :
    - df : DataFrame à afficher
    - height : hauteur (en pixels) de la boîte (par défaut 300)
    - border_color : couleur de la bordure (par défaut lightgray)
    - padding : espacement intérieur (en pixels, par défaut 10)
    """
    html = df.to_html(index=index)
    return HTML(
        f"""
        <div style="height:{height}px; overflow:auto;
                    border:1px solid {border_color}; padding:{padding}px">
            {html}
        </div>
        """
    )

In [ ]:
X_train_clean = X_train.copy()

## Répartition des types de variables dans `X_train`

L’inspection des types de données permet d’identifier la nature des variables présentes dans le jeu `X_train`.
Cette étape est essentielle pour distinguer :

- Les variables numériques (quantitatives) ;
- Les variables catégorielles (qualitatives) ;
- D’éventuelles valeurs mal typées, comme par exemple des colonnes numériques encodées en `object` à cause de formats hétérogènes.

Cette classification est indispensable pour anticiper les traitements appropriés lors de la phase de prétraitement : encodage, imputation, normalisation, etc.

Affichons les types de données de `X_train` ainsi que le nombre de colonnes associées à chaque type.

In [ ]:
display(X_train.dtypes.value_counts())

La répartition des types de variables dans `X_train` est la suivante :

- 280 variables de type `object` : ces colonnes correspondent à des données catégorielles ou textuelles. Elles nécessitent un traitement spécifique (encodage, nettoyage) avant utilisation dans un modèle ;
- 58 variables de type `int64` : généralement des entiers utilisés comme indicateurs, compteurs ou encodage de modalités ;
- 36 variables de type `float64` : variables numériques continues, souvent associées à des mesures (ex. : surface, capitaux, indices).

Après avoir identifié la nature des variables dans `X_train`, une première étape de nettoyage consiste à supprimer les variables non informatives.
Ces variables, sans réelle valeur ajoutée pour l'apprentissage, peuvent altérer la performance des modèles ou en complexifier inutilement l'entraînement.

Nous procéderons selon les critères suivants, dans cet ordre :
- Colonnes quasi vides : contenant un taux élevé de valeurs manquantes, rendant leur interprétation incertaine ;
- Colonnes constantes : ne présentant qu’une seule modalité, et donc aucune variabilité ;
- Colonnes déséquilibrées : présentant une modalité dominante écrasante (ex. > 99 %) ;
- Colonnes identifiantes : contenant des identifiants uniques (ex. ID) sans relation directe avec la variable cible.

Cette approche graduelle permet de rationaliser le jeu de données tout en préservant l’information pertinente pour la modélisation.

## Détection et suppression des variables non informatives

### Analyse des valeurs manquantes

L’analyse des valeurs manquantes permet d’évaluer la qualité du jeu de données en identifiant les variables partiellement renseignées. Ces colonnes devront faire l’objet d’un traitement spécifique : suppression, imputation ou méthodes adaptées au contexte.

Pour chaque variable, seront indiqués :

- Le **nombre de valeurs manquantes** ;
- La **proportion de valeurs manquantes** par rapport au total d’observations ;
- Un **tri décroissant** selon cette proportion, afin de faire ressortir les variables les plus concernées.

In [ ]:
def summarize_missing_values(df, sort=True, threshold=0):
    """
    Résume le nombre et le taux de valeurs manquantes pour chaque colonne
    d'un DataFrame.

    Paramètres :
    -----------
    df : pd.DataFrame
        Le DataFrame à analyser.

    sort : bool (par défaut True)
        Si True, trie les colonnes par taux de valeurs manquantes décroissant.

    threshold : int (par défaut 0)
        Ne conserve que les colonnes avec plus de `threshold` valeurs
        manquantes.

    display_func : callable (par défaut None)
        Fonction pour afficher le DataFrame résultant
        (ex : display_scrollable_df).

    Retour :
    --------
    missing_info : pd.DataFrame
        Tableau récapitulatif des valeurs manquantes.
    """
    try:
        nb_missing = df.isnull().sum()
        taux_missing = df.isnull().mean() * 100

        missing_info = pd.DataFrame(
            {
                "Variable": df.columns,
                "Valeurs manquantes": nb_missing,
                "Taux (%)": taux_missing.round(2),
            }
        )

        # Filtrage
        missing_info = missing_info[missing_info["Valeurs manquantes"] >= 0]

        # Tri
        if sort:
            missing_info = missing_info.sort_values(
                by="Taux (%)", ascending=False
            )

        return missing_info
    except Exception as e:
        print(e)

In [ ]:
missing_info_X_train = summarize_missing_values(
    X_train, threshold=0, sort=True
)
display_scrollable_df(missing_info_X_train)

Le tableau obtenu met en évidence une proportion importante de variables avec des taux de valeurs manquantes significatifs. Ces variables peuvent être classées selon différents niveaux d’impact :

- **Totalement ou quasi totalement manquantes** : `DEROG14` (100 %), `DEROG13` (99,66 %), `DEROG16` (99,15 %), `CARACT2` (96,00 %), `CARACT3` (96,00 %), `TYPBAT1` (92,11 %), `DEROG12` (91,00 %)
- **Taux très élevés (> 90 %)** : `CARACT2`, `CARACT3`, `TYPBAT1`, `DEROG12`
- **Taux intermédiaires (56–62 %)** : variables météo et distances (`RRAB_VOR_MM_A`, `DISTANCE_311`, etc.)
- **Taux modérés (20–40 %)** : `RISK13`, `KAPITAL11`, `SURFACE8`, etc.
- **Taux faibles (< 10 %)** : `MEN_MAIS`, `RISK1`, `FRCH2`, etc.

> **Remarques méthodologiques :**
> - Les variables affichant plus de **90 % de valeurs manquantes** sont de sérieuses candidates à l’exclusion.
> - Certaines **familles de variables cohérentes** (distances, météo) justifient une réflexion approfondie :
>   - Imputation croisée entre variables similaires ;
>   - Imputation conditionnelle selon le contexte géographique ou sectoriel ;
>   - Réduction de dimension (agrégation, ACP).

#### Suppression des colonnes quasi vides (trop de valeurs manquantes)

En se basant sur l’analyse précédente, les colonnes présentant plus de **90 % de valeurs manquantes** sont supprimées du jeu de données `X_train_clean`.
Cette étape permet de réduire le bruit et la complexité du modèle en éliminant les variables qui n’apportent pas d’information significative.

In [ ]:
def drop_mostly_empty_columns(df, seuil=0.9, verbose=False):
    """
    Supprime les colonnes contenant une proportion de valeurs manquantes
    supérieure à un seuil donné.

    Paramètres :
    - df : DataFrame pandas d'entrée.
    - seuil : float, seuil de tolérance (ex. 0.9 pour 90% de
        valeurs manquantes).
    - verbose : bool, affiche ou non les informations de suppression.

    Retourne :
    - df_clean : DataFrame nettoyé.
    - colonnes_supprimées : liste des noms des colonnes supprimées.
    """
    missing_ratio = df.isnull().mean()
    cols_to_drop = missing_ratio[missing_ratio > seuil].index.tolist()
    df_clean = df.drop(columns=cols_to_drop)

    if verbose:
        print(
            f"Suppression de {len(cols_to_drop)} colonnes avec plus de "
            f"{seuil*100:.0f}% de valeurs manquantes :\n"
            f"{cols_to_drop}\n"
            f"Nombre de colonnes restantes : {df_clean.shape[1]}"
        )

    return df_clean, cols_to_drop

In [ ]:
X_train_clean, empty_columns = drop_mostly_empty_columns(
    X_train, seuil=0.9, verbose=True
)

Après la suppression des colonnes avec plus de 90% de valeurs manquantes (`CARACT2`, `CARACT3`, `TYPBAT1`, `DEROG12`, `DEROG13`, `DEROG14`, `DEROG16`), le jeu de données `X_train_clean` contient désormais **367 colonnes**. Cette réduction permet de se concentrer sur les variables les plus pertinentes pour la modélisation.
Affichons le nombre de colonnes restantes après cette première étape de nettoyage.

In [ ]:
missing_info_X_train_clean = summarize_missing_values(
    X_train_clean, threshold=0, sort=True
)
display_scrollable_df(missing_info_X_train_clean)

La suppression des colonnes quasi vides réalisée, supprimons à présent les colonnes constantes, c’est-à-dire celles qui ne présentent qu’une seule modalité (valeur unique) sur l’ensemble des observations. Ces variables n’apportent aucune information discriminante pour la modélisation et peuvent être éliminées.

#### Suppression des colonnes constantes

In [ ]:
def drop_constant_columns(df, verbose=False):
    """
    Suppression des colonnes décrites par une modalité uniques.

    Parameters:
        - df (pd.DataFrame): DataFrame pandas d'entrée.
        - verbose : bool, affiche ou non les informations de suppression.

    Returns:
        pd.DataFrame: Nouvelle DataFrame sans les colonnes constantes.
    """
    # Identification des colonnes constantes
    # dropna=True permet de supprimer les valeurs type NAN et ainsi se
    # focaliser exclusivement sur les valeurs exploitables
    cols_constant = [
        col for col in df.columns if df[col].nunique(dropna=True) <= 1
    ]
    df_clean = df.drop(columns=cols_constant)
    if verbose:
        if not cols_constant:
            print("Aucune colonne constante n'a été trouvée.")
        else:
            print(
                f"Les {len(cols_constant)} colonnes constantes "
                f"ont été supprimées : {cols_constant}"
            )

    # Retourne la DataFrame sans les colonnes constantes
    return df_clean, cols_constant

In [ ]:
X_train_clean, constant_columns = drop_constant_columns(
    X_train_clean, verbose=True
)

Les colonnes `DISTANCE_244`, `IND_Y1_Y2` et `IND_INC` ne prenant qu'une seule modalité, ont été supprimées.
Abordons à présent la question des variables déséquilibrées, c’est-à-dire celles qui présentent une modalité dominante écrasante.

#### Colonnes avec une modalité ultra-dominante

In [ ]:
def drop_dominant_modalities(df, threshold=0.99, verbose=False):
    """
    Supprime les colonnes où une seule modalité représente plus
    de `threshold` % des valeurs.

    Paramètres :
        df (pd.DataFrame) : Le DataFrame à traiter.
        threshold (float) : Seuil de dominance (par défaut : 0.99 pour 99 %).

    Retour :
        pd.DataFrame : Le DataFrame nettoyé.
    """
    cols_to_drop = [
        col
        for col in df.columns
        if df[col].value_counts(normalize=True, dropna=False).values[0]
        > threshold
    ]
    if verbose:
        print(
            f"{len(cols_to_drop)} colonne(s) avec une "
            f"modalité dominante > {threshold*100:.0f}% supprimée(s) :\n"
            f"{cols_to_drop}"
        )

    return df.drop(columns=cols_to_drop)

In [ ]:
X_train_clean = drop_dominant_modalities(
    X_train_clean, threshold=0.99, verbose=True
)

L'étape de suppression des colonnes avec une modalité ultra-dominante réalisée, supprimons à présent les colonnes relatives aux identifiants des données : `ID`.

#### Suppression des colonnes identifiantes

In [ ]:
# suppression de la colonne ID
cols_id = ["ID"]
X_train_clean.drop(columns=cols_id, inplace=True)

Pour clôturer cette phase de nettoyage, comparons la répartition des types de variables entre `X_train` et `X_train_clean`, en mettant en évidence le nombre de colonnes supprimées pour chaque type.

In [ ]:
# Nombre de colonnes par type de données
types_clean = X_train_clean.dtypes.value_counts().rename("X_train_clean")
types_raw = X_train.dtypes.value_counts().rename("X_train")

# Concaténation
types_comparison = (
    pd.concat([types_raw, types_clean], axis=1).fillna(0).astype(int)
)

# Ajout de la différence (colonnes supprimées)
types_comparison["Nombre de colonnes supprimées"] = (
    types_comparison["X_train"] - types_comparison["X_train_clean"]
)

# Tri par nom de type pour cohérence
types_comparison = types_comparison.sort_index()

display(types_comparison)

Le tableau ci-dessous synthétise les différentes suppressions de variables jugées non informatives au regard des critères définis précédemment. 

| Critère de suppression                                 | Nombre de colonnes | Colonnes supprimées                                                                 |
|--------------------------------------------------------|--------------------|--------------------------------------------------------------------------------------|
| > 90 % de valeurs manquantes                           | 7                  | `CARACT2`, `CARACT3`, `TYPBAT1`, `DEROG12`, `DEROG13`, `DEROG14`, `DEROG16`         |
| Constantes (une seule modalité)                        | 3                  | `DISTANCE_244`, `IND_Y1_Y2`, `IND_INC`                                              |
| Modalité dominante > 99 %                              | 23                 | `DEROG1`, `DEROG6`, `DEROG7`, `DEROG9`, `DEROG10`, `DEROG15`, `CA2`, `KAPITAL2`, `KAPITAL4`, `KAPITAL5`, `KAPITAL15`, `KAPITAL18`, `KAPITAL20`, `KAPITAL36`, `KAPITAL38`, `KAPITAL39`, `SURFACE14`, `SURFACE15`, `SURFACE20`, `NBBAT3`, `NBBAT5`, `NBBAT7`, `NBBAT11`  |
| Identifiants (colonne ID)                              | 1                  | `ID`                                                                                 |

> **Total supprimé** : **34 colonnes**

Ces suppressions permettent d’alléger le jeu de données tout en conservant l’information utile à la modélisation.

Les variables non informatives écartées, il convient désormais d’approfondir l’analyse de la variabilité interne de celles restantes.

Cette étape vise à évaluer le nombre de modalités distinctes par variable, un indicateur crucial pour orienter les choix d’encodage, de regroupement ou de réduction de cardinalité dans la suite du prétraitement.

## Analyse du nombre de modalités par variable

À la suite de l’identification et de la suppression des variables non informatives (constantes, quasi vides ou dominées par une seule modalité), il est désormais pertinent d’examiner plus finement la variabilité des modalités au sein de chaque variable.

Cette analyse constitue une étape clef pour :

- Quantifier la diversité des modalités (cardinalité) par variable ;
- Repérer les variables peu informatives (1 à 2 modalités) ;
- Identifier les variables à cardinalité élevée, qui peuvent complexifier l’encodage ;
- Orienter les choix d’encodage (one-hot, ordinal, fréquentiel, binning, etc.) ;
- Détecter d’éventuelles colonnes mal typées ou numériques discrètes.

Pour faciliter la lecture, les variables sont regroupées par tranches de cardinalité :

- $\leq$ 2 modalités : souvent binaires ou très peu informatives ;
- 3–5 modalités : typiques des variables catégorielles simples ;
- 6–10 modalités : à surveiller pour un encodage spécifique ou une agrégation ;
- $\gt$ 10 modalités : nécessitent un traitement spécifique pour éviter la surcharge lors de l'encodage.

> Ce regroupement servira de base pour définir les traitements à appliquer, entre exclusion, transformation, ou encodage optimisé.

L'étape suivante consiste donc à afficher et analyser le nombre de valeurs uniques pour chaque variable du jeu X_train, en les classant selon ces tranches.

In [ ]:
def grouped_modalities(df):
    # Calcul du nombre de modalités uniques par variable
    modalities = df.nunique()

    # Construction du DataFrame
    df_modalities = pd.DataFrame(
        {"Variable": modalities.index, "Modalités uniques": modalities.values}
    )

    # Tri alphabétique des noms de variables
    df_modalities = df_modalities.sort_values(by="Variable")

    # Regroupement par nombre de modalités uniques
    return (
        df_modalities.groupby("Modalités uniques")["Variable"]
        .apply(list)
        .reset_index()
    )


grouped_modalities = grouped_modalities(X_train_clean)
# Affichage
display_scrollable_df(grouped_modalities)

L’analyse du nombre de valeurs distinctes par variable permet d’anticiper les traitements nécessaires avant la phase de modélisation : encodage, regroupement, réduction de dimension ou exclusion.

| Modalités uniques | Variables |
|-------------------|-----------|
| 2 | `ADOSS`, `DEROG11`, `DEROG2`, `DEROG3`, `DEROG4`, `DEROG5`, `DEROG8`, `EQUIPEMENT1`, `EQUIPEMENT4`, `INDEM1`, `IND_0_Y1`, `IND_Y2_Y3`, `IND_Y3_Y4`, `IND_Y4_Y5`, `KAPITAL1`, `KAPITAL3`, `KAPITAL34`, `KAPITAL35`, `KAPITAL37`, `KAPITAL40`, `KAPITAL41`, `KAPITAL42`, `KAPITAL43`, `KAPITAL6`, `KAPITAL7`, `KAPITAL8`, `KAPITAL9`, `LOG_INC`, `RISK8`, `TYPERS` |
| 3 | `CARACT1`, `DISTANCE_2`, `EQUIPEMENT2`, `IND`, `IND_Y7_Y8`, `IND_Y9`, `KAPITAL13`, `KAPITAL27`, `KAPITAL28`, `KAPITAL29`, `KAPITAL30`, `KAPITAL31`, `MEN`, `MEN_5IND`, `MEN_FMP`, `RISK10`, `RISK11`, `RISK12`, `RISK13`, `RISK6`, `RISK9`, `TYPBAT2`, `ZONE_VENT` |
| 4 | `ALTITUDE_1`, `ALTITUDE_2`, `ALTITUDE_3`, `ALTITUDE_4`, `ALTITUDE_5`, `BDTOPO_BAT_MAX_HAUTEUR`, `BDTOPO_BAT_MAX_HAUTEUR_MAX`, `DISTANCE_1`, `DISTANCE_111`, `DISTANCE_112`, `DISTANCE_121`, `DISTANCE_122`, `DISTANCE_123`, `DISTANCE_124`, `DISTANCE_131`, `DISTANCE_132`, `DISTANCE_133`, `DISTANCE_141`, `DISTANCE_142`, `DISTANCE_211`, `DISTANCE_212`, `DISTANCE_213`, `DISTANCE_221`, `DISTANCE_222`, `DISTANCE_223`, `DISTANCE_231`, `DISTANCE_242`, `DISTANCE_243`, `DISTANCE_321`, `DISTANCE_322`, `DISTANCE_323`, `DISTANCE_331`, `DISTANCE_332`, `DISTANCE_333`, `DISTANCE_334`, `DISTANCE_335`, `DISTANCE_411`, `DISTANCE_412`, `DISTANCE_421`, `DISTANCE_422`, `DISTANCE_423`, `DISTANCE_511`, `DISTANCE_512`, `DISTANCE_521`, `DISTANCE_522`, `DISTANCE_523`, `DUREE_REQANEUF`, `ESPINSEE`, `FFM_VOR_COM_MMAX_A_Y`, `FFM_VOR_COM_MM_A_Y`, `FFM_VOR_MMAX_A`, `FFM_VOR_MM_A`, `FXI3SAB_VOR_COM_MMAX_A_Y`, `FXI3SAB_VOR_COM_MM_A_Y`, `FXI3SAB_VOR_MMAX_A`, `FXI3SAB_VOR_MM_A`, `FXIAB_VOR_MMAX_A`, `FXIAB_VOR_MM_A`, `FXYAB_VOR_MMAX_A`, `FXYAB_VOR_MM_A`, `HAUTEUR`, `HAUTEUR_MAX`, `IND_SNV`, `IND_Y5_Y6`, `IND_Y6_Y7`, `KAPITAL14`, `MEN_SURF`, `NBJFF10_MMAX_A`, `NBJFF10_MM_A`, `NBJFF10_MSOM_A`, `NBJFF16_MMAX_A`, `NBJFF16_MM_A`, `NBJFF16_MSOM_A`, `NBJFF28_MMAX_A`, `NBJFF28_MM_A`, `NBJFF28_MSOM_A`, `NBJFXI3S10_MMAX_A`, `NBJFXI3S10_MM_A`, `NBJFXI3S10_MSOM_A`, `NBJFXI3S16_MMAX_A`, `NBJFXI3S16_MM_A`, `NBJFXI3S16_MSOM_A`, `NBJFXI3S28_MMAX_A`, `NBJFXI3S28_MM_A`, `NBJFXI3S28_MSOM_A`, `NBJFXY10_MMAX_A`, `NBJFXY10_MM_A`, `NBJFXY10_MSOM_A`, `NBJFXY15_MMAX_A`, `NBJFXY15_MM_A`, `NBJFXY15_MSOM_A`, `NBJFXY8_MMAX_A`, `NBJFXY8_MM_A`, `NBJFXY8_MSOM_A`, `NBJRR100_MMAX_A`, `NBJRR100_MM_A`, `NBJRR100_MSOM_A`, `NBJRR10_MMAX_A`, `NBJRR10_MM_A`, `NBJRR10_MSOM_A`, ... |
| 5 | `CA3`, `DISTANCE_311`, `DISTANCE_312`, `DISTANCE_313`, `DISTANCE_324`, `KAPITAL26`, `PROPORTION_13` |
| 6 | `CARACT4`, `COEFASS`, `FRCH2`, `IND_Y8_Y9`, `KAPITAL16`, `KAPITAL17`, `KAPITAL24`, `MEN_PAUV`, `PROPORTION_41`, `PROPORTION_42`, `SURFACE21` |
| 7 | `LOG_A1_A2`, `LOG_SOC`, `MEN_1IND`, `PROPORTION_12`, `PROPORTION_51`, `PROPORTION_52`, `RISK7`, `SURFACE12`, `SURFACE16`, `SURFACE17` |
| 8 | `CA1`, `EQUIPEMENT5`, `MEN_PROP`, `SURFACE9`, `VOCATION` |
| 9 | `ACTIVIT2`, `AN_EXERC`, `FRCH1`, `LOG_A2_A3`, `PROPORTION_32`, `PROPORTION_33` |
| 10 | `EQUIPEMENT3`, `KAPITAL32`, `KAPITAL33`, `LOG_APA3`, `LOG_AVA1`, `MEN_COLL`, `MEN_MAIS`, `PROPORTION_11`, `PROPORTION_21`, `PROPORTION_22`, `PROPORTION_23`, `PROPORTION_24`, `PROPORTION_31`, `TAILLE1`, `TAILLE2` |
| 11 | `KAPITAL19`, `KAPITAL21`, `KAPITAL22`, `KAPITAL23`, `NBBAT14`, `NBBAT6`, `NBBAT8`, `RISK1` |
| 12 | `ANCIENNETE`, `INDEM2`, `RISK4`, `RISK5`, `SURFACE19`, `TAILLE4` |
| 13 | `KAPITAL10`, `KAPITAL11`, `KAPITAL12`, `SURFACE13` |
| 16 | `CARACT5`, `NBBAT2`, `NBBAT9`, `SURFACE4`, `SURFACE6` |
| 17 | `TAILLE3` |
| 19 | `SURFACE10`, `SURFACE7` |
| 21 | `KAPITAL25`, `NBBAT10` |
| 22 | `EQUIPEMENT6`, `EQUIPEMENT7` |
| 27 | `RISK3` |
| 30 | `NBBAT1` |
| 31 | `NBBAT13`, `NBBAT4` |
| 49 | `RISK2` |
| 53 | `NBSINCONJ` |
| 61 | `SURFACE18` |
| 65 | `SURFACE1`, `SURFACE11`, `SURFACE2`, `SURFACE3`, `SURFACE5`, `SURFACE8` |
| 93 | `ZONE` |
| 94 | `NBSINSTRT` |
| 1113 | `ANNEE_ASSURANCE` |

À partir du tableau précédent, qui répertorie les variables selon leur nombre de modalités uniques, plusieurs stratégies de traitement peuvent être envisagées : encodage, regroupement ou transformation. Pour faciliter ces opérations, les variables sont segmentées par tranches de cardinalité à l’aide de la méthode `group_variables_by_cardinality`.

In [ ]:
def group_variables_by_cardinality(df, thresholds=None):
    """
    Groups DataFrame variables by cardinality intervals
    (number of unique values),
    with alphabetical sorting within each group.
    All variables are fully displayed.

    Parameters:
    - df : DataFrame to analyze
    - thresholds : list of upper bounds (exclusive) for cardinality bins.
                   Default is [2, 5, 10, 30, 100, float('inf')]

    Returns:
    - grouped_df : DataFrame with columns ["Unique Values", "Variables"]
    """
    if thresholds is None:
        thresholds = [2, 5, 10, 30, 100, float("inf")]

    # Count unique values per variable
    n_unique = df.nunique().sort_index()
    grouped = []

    for i, threshold in enumerate(thresholds):
        min_val = 0 if i == 0 else thresholds[i - 1] + 1
        max_val = threshold

        # Filter variables that fall within the interval
        vars_in_range = n_unique[
            (n_unique >= min_val) & (n_unique <= max_val)
        ].sort_index()

        if not vars_in_range.empty:
            # Fully join variables with backticks for clarity
            var_list = ", ".join(f"{v}" for v in sorted(vars_in_range.index))
            grouped.append(
                {
                    "Unique Values": (
                        f"{min_val}-{max_val}"
                        if max_val != float("inf")
                        else f">{min_val - 1}"
                    ),
                    "Variables": var_list,
                }
            )

    return pd.DataFrame(grouped)

In [ ]:
df_card_summary = group_variables_by_cardinality(X_train_clean)
display_scrollable_df(df_card_summary)

Le tableau ci-dessous récapitule les variables, leurs intervalles de modalités uniques associés et les méthodes envisagées.

| Intervalles de modalités uniques | Variables | Méthodes recommandées | Organisation |
|---------------|-----------| --------------------------------------------------- |-----------|
| 0–2 | `ADOSS`, `DEROG11`, `DEROG2`, `DEROG3`, `DEROG4`, `DEROG5`, `DEROG8`, `EQUIPEMENT1`, `EQUIPEMENT4`, `INDEM1`, `IND_0_Y1`, `IND_Y2_Y3`, `IND_Y3_Y4`, `IND_Y4_Y5`, `KAPITAL1`, `KAPITAL3`, `KAPITAL34`, `KAPITAL35`, `KAPITAL37`, `KAPITAL40`, `KAPITAL41`, `KAPITAL42`, `KAPITAL43`, `KAPITAL6`, `KAPITAL7`, `KAPITAL8`, `KAPITAL9`, `LOG_INC`, `RISK8`, `TYPERS` | `Label encoding` si les modalités sont ordinales / Transformation en booléen si binaire | Identifier rapidement les constantes (à éliminer si nécessaire) / Appliquer le mapping ou transformation binaire|
| 3–5 | `ALTITUDE_1`, `ALTITUDE_2`, `ALTITUDE_3`, `ALTITUDE_4`, `ALTITUDE_5`, `BDTOPO_BAT_MAX_HAUTEUR`, `BDTOPO_BAT_MAX_HAUTEUR_MAX`, `CA3`, `CARACT1`, `DISTANCE_1`, `DISTANCE_111`, `DISTANCE_112`, `DISTANCE_121`, `DISTANCE_122`, `DISTANCE_123`, `DISTANCE_124`, `DISTANCE_131`, `DISTANCE_132`, `DISTANCE_133`, `DISTANCE_141`, `DISTANCE_142`, `DISTANCE_2`, `DISTANCE_211`, `DISTANCE_212`, `DISTANCE_213`, `DISTANCE_221`, `DISTANCE_222`, `DISTANCE_223`, `DISTANCE_231`, `DISTANCE_242`, `DISTANCE_243`, `DISTANCE_311`, `DISTANCE_312`, `DISTANCE_313`, `DISTANCE_321`, `DISTANCE_322`, `DISTANCE_323`, `DISTANCE_324`, `DISTANCE_331`, `DISTANCE_332`, `DISTANCE_333`, `DISTANCE_334`, `DISTANCE_335`, `DISTANCE_411`, `DISTANCE_412`, `DISTANCE_421`, `DISTANCE_422`, `DISTANCE_423`, `DISTANCE_511`, `DISTANCE_512`, `DISTANCE_521`, `DISTANCE_522`, `DISTANCE_523`, `DUREE_REQANEUF`, `EQUIPEMENT2`, `ESPINSEE`, `FFM_VOR_COM_MMAX_A_Y`, `FFM_VOR_COM_MM_A_Y`, `FFM_VOR_MMAX_A`, `FFM_VOR_MM_A`, `FXI3SAB_VOR_COM_MMAX_A_Y`, `FXI3SAB_VOR_COM_MM_A_Y`, `FXI3SAB_VOR_MMAX_A`, `FXI3SAB_VOR_MM_A`, `FXIAB_VOR_MMAX_A`, `FXIAB_VOR_MM_A`, `FXYAB_VOR_MMAX_A`, `FXYAB_VOR_MM_A`, `HAUTEUR`, `HAUTEUR_MAX`, `IND`, `IND_SNV`, `IND_Y5_Y6`, `IND_Y6_Y7`, `IND_Y7_Y8`, `IND_Y9`, `KAPITAL13`, `KAPITAL14`, `KAPITAL26`, `KAPITAL27`, `KAPITAL28`, `KAPITAL29`, `KAPITAL30`, `KAPITAL31`, `MEN`, `MEN_5IND`, `MEN_FMP`, `MEN_SURF`, `NBJFF10_MMAX_A`, `NBJFF10_MM_A`, `NBJFF10_MSOM_A`, `NBJFF16_MMAX_A`, `NBJFF16_MM_A`, `NBJFF16_MSOM_A`, `NBJFF28_MMAX_A`, `NBJFF28_MM_A`, `NBJFF28_MSOM_A`, `NBJFXI3S10_MMAX_A`, `NBJFXI3S10_MM_A`, `NBJFXI3S10_MSOM_A`, `NBJFXI3S16_MMAX_A`, `NBJFXI3S16_MM_A`, `NBJFXI3S16_MSOM_A`, `NBJFXI3S28_MMAX_A`, `NBJFXI3S28_MM_A`, `NBJFXI3S28_MSOM_A`, `NBJFXY10_MMAX_A`, `NBJFXY10_MM_A`, `NBJFXY10_MSOM_A`, `NBJFXY15_MMAX_A`, `NBJFXY15_MM_A`, `NBJFXY15_MSOM_A`, `NBJFXY8_MMAX_A`, `NBJFXY8_MM_A`, `NBJFXY8_MSOM_A`, `NBJRR100_MMAX_A`, `NBJRR100_MM_A`, `NBJRR100_MSOM_A`, `NBJRR10_MMAX_A`, `NBJRR10_MM_A`, `NBJRR10_MSOM_A`, `NBJRR1_MMAX_A`, `NBJRR1_MM_A`, `NBJRR1_MSOM_A`, `NBJRR30_MMAX_A`, `NBJRR30_MM_A`, `NBJRR30_MSOM_A`, `NBJRR50_MMAX_A`, `NBJRR50_MM_A`, `NBJRR50_MSOM_A`, `NBJRR5_MMAX_A`, `NBJRR5_MM_A`, `NBJRR5_MSOM_A`, `NBJTMS24_MMAX_A`, `NBJTMS24_MM_A`, `NBJTMS24_MSOM_A`, `NBJTN10_MMAX_A`, `NBJTN10_MM_A`, `NBJTN10_MSOM_A`, `NBJTN5_MMAX_A`, `NBJTN5_MM_A`, `NBJTN5_MSOM_A`, `NBJTNI10_MMAX_A`, `NBJTNI10_MM_A`, `NBJTNI10_MSOM_A`, `NBJTNI15_MMAX_A`, `NBJTNI15_MM_A`, `NBJTNI15_MSOM_A`, `NBJTNI20_MMAX_A`, `NBJTNI20_MM_A`, `NBJTNI20_MSOM_A`, `NBJTNS20_MMAX_A`, `NBJTNS20_MM_A`, `NBJTNS20_MSOM_A`, `NBJTNS25_MMAX_A`, `NBJTNS25_MM_A`, `NBJTNS25_MSOM_A`, `NBJTX0_MMAX_A`, `NBJTX0_MM_A`, `NBJTX0_MSOM_A`, `NBJTX25_MMAX_A`, `NBJTX25_MM_A`, `NBJTX25_MSOM_A`, `NBJTX30_MMAX_A`, `NBJTX30_MM_A`, `NBJTX30_MSOM_A`, `NBJTX35_MMAX_A`, `NBJTX35_MM_A`, `NBJTX35_MSOM_A`, `NBJTXI20_MMAX_A`, `NBJTXI20_MM_A`, `NBJTXI20_MSOM_A`, `NBJTXI27_MMAX_A`, `NBJTXI27_MM_A`, `NBJTXI27_MSOM_A`, `NBJTXS32_MMAX_A`, `NBJTXS32_MM_A`, `NBJTXS32_MSOM_A`, `NB_CASERNES`, `PROPORTION_13`, `PROPORTION_14`, `RISK10`, `RISK11`, `RISK12`, `RISK13`, `RISK6`, `RISK9`, `RRAB_VOR_MMAX_A`, `RRAB_VOR_MM_A`, `RR_VOR_MMAX_A`, `RR_VOR_MM_A`, `TAMPLIAB_VOR_MMAX_A`, `TAMPLIAB_VOR_MM_A`, `TAMPLIM_VOR_MMAX_A`, `TAMPLIM_VOR_MM_A`, `TMMAX_VOR_MMAX_A`, `TMMAX_VOR_MM_A`, `TMMIN_VOR_MMAX_A`, `TMMIN_VOR_MM_A`, `TMM_VOR_MMAX_A`, `TMM_VOR_MM_A`, `TM_VOR_MMAX_A`, `TM_VOR_MM_A`, `TNAB_VOR_MMAX_A`, `TNAB_VOR_MM_A`, `TNMAX_VOR_MMAX_A`, `TNMAX_VOR_MM_A`, `TN_VOR_MMAX_A`, `TN_VOR_MM_A`, `TXAB_VOR_MMAX_A`, `TXAB_VOR_MM_A`, `TXMIN_VOR_MMAX_A`, `TXMIN_VOR_MM_A`, `TX_VOR_MMAX_A`, `TX_VOR_MM_A`, `TYPBAT2`, `ZONE_VENT` | `One-hot encoding` ou `Ordinal encoding` / Fusionner les modalités rares si besoin | Vérifier la distribution des modalités / Décider selon la sémantique si un ordre est présent (ordinal) ou non (nominal) |
| 6–10 | `ACTIVIT2`, `AN_EXERC`, `CA1`, `CARACT4`, `COEFASS`, `EQUIPEMENT3`, `EQUIPEMENT5`, `FRCH1`, `FRCH2`, `IND_Y8_Y9`, `KAPITAL16`, `KAPITAL17`, `KAPITAL24`, `KAPITAL32`, `KAPITAL33`, `LOG_A1_A2`, `LOG_A2_A3`, `LOG_APA3`, `LOG_AVA1`, `LOG_SOC`, `MEN_1IND`, `MEN_COLL`, `MEN_MAIS`, `MEN_PAUV`, `MEN_PROP`, `PROPORTION_11`, `PROPORTION_12`, `PROPORTION_21`, `PROPORTION_22`, `PROPORTION_23`, `PROPORTION_24`, `PROPORTION_31`, `PROPORTION_32`, `PROPORTION_33`, `PROPORTION_41`, `PROPORTION_42`, `PROPORTION_51`, `PROPORTION_52`, `RISK7`, `SURFACE12`, `SURFACE16`, `SURFACE17`, `SURFACE21`, `SURFACE9`, `TAILLE1`, `TAILLE2`, `VOCATION` | Ordinal / Fréquentiel / Binning si ordinal          | Vérifier la nature des modalités (ordonnées ou non) / Inspecter les fréquences pour évaluer les regroupements pertinents |
| 11–30 | `ANCIENNETE`, `CARACT5`, `EQUIPEMENT6`, `EQUIPEMENT7`, `INDEM2`, `KAPITAL10`, `KAPITAL11`, `KAPITAL12`, `KAPITAL19`, `KAPITAL21`, `KAPITAL22`, `KAPITAL23`, `KAPITAL25`, `NBBAT1`, `NBBAT10`, `NBBAT14`, `NBBAT2`, `NBBAT6`, `NBBAT8`, `NBBAT9`, `RISK1`, `RISK3`, `RISK4`, `RISK5`, `SURFACE10`, `SURFACE13`, `SURFACE19`, `SURFACE4`, `SURFACE6`, `SURFACE7`, `TAILLE3`, `TAILLE4` | Regroupement par thème ou par fréquence / `Target encoding` (avec précaution, en cross-validation) | Étudier les significations des modalités pour des regroupements manuels / Vérifier le risque de surapprentissage avec des méthodes supervisées |
| 31–100 | `NBBAT13`, `NBBAT4`, `NBSINCONJ`, `NBSINSTRT`, `RISK2`, `SURFACE1`, `SURFACE11`, `SURFACE18`, `SURFACE2`, `SURFACE3`, `SURFACE5`, `SURFACE8`, `ZONE` | Fusion, Encodage fréquentiel, Clustering | Évaluer si certaines modalités sont dominantes ou inutiles / Réduire la cardinalité par agrégation thématique |
| >100 | `ANNEE_ASSURANCE` | *Binning*, normalisation, réduction (*PCA*, *clustering*, *UMAP*) | Analyser s’il s’agit de variables continues ou mal encodées / Appliquer des techniques de compression de l'information|

Préalablement, une méthode `extract_variables_from_interval` est implémenté pour extraire les variables associés aux intervalles décrits ci-dessus.

In [ ]:
def extract_variables_from_interval(df, interval):
    # Extraction de la chaîne de variables pour l'intervalle donné
    variables_str = df.loc[
        df["Unique Values"] == interval, "Variables"
    ].values[0]

    # Conversion de la chaîne en liste de variables
    return [v.strip() for v in variables_str.split(",")]

Afin d’identifier les modalités associées à chaque variable étudiée, deux méthodes ont été implémentées :

- `detect_variable_type` permettant d'extraire la typologie des modalités associées à chacune des variables ;
- `get_unique_values_per_variable` permettant d’extraire, à partir d’un *DataFrame* donné, l’ensemble des modalités uniques propres à chaque variable.

In [ ]:
def detect_variable_type(modalites):
    modalites = str(modalites)

    # Check for clearly numeric thresholds
    if re.search(r"<=|>=", modalites):
        if re.search(r"\d", modalites):
            return "Continue discrétisée"

    # Check for categorical strings
    if re.fullmatch(
        r"\[('[A-Z0-9_]+',?\s?)+\]",
        modalites.replace(" ", "").replace('"', "'"),
    ):
        return "Nominale"

    # Detect purely string-based categories
    if all(
        re.match(r"[A-Z]", m.strip())
        for m in re.sub(r"[\[\]]", "", modalites).split(",")
    ):
        return "Nominale"

    # If all modalities are numerical cutoffs but very few
    if re.search(r"^[\[\(].*[\]\)]$", modalites) or re.search(
        r"\d", modalites
    ):
        return "Ordinale"

    return "Inconnue"

In [ ]:
def get_unique_values_per_variable(df, list_vars):
    """
    Renvoie un DataFrame avec les modalités uniques pour chaque variable.

    Paramètres :
    - df : DataFrame source
    - list_vars : liste des noms de colonnes (chaînes)

    Retour :
    - DataFrame avec les colonnes :
        - "Variable" ;
        - "Modalités" ;
        - "Nombre de Modalités" ;
        - "Typologie détectée".
    """
    data = {
        "Variable": [],
        "Modalités": [],
        "Nombre de Modalités": [],
        "Typologie détectée": [],
    }

    for var in list_vars:
        if var in df.columns:
            uniques = sorted(df[var].dropna().unique().tolist())
            data["Variable"].append(var)
            data["Modalités"].append(uniques)
            data["Nombre de Modalités"].append(len(uniques))
            data["Typologie détectée"].append(detect_variable_type(uniques))

    return pd.DataFrame(data)

Pour chaque intervalle et pour l'ensemble des variables associées, le traitement se fera selon les étapes décrites dans le tableau ci-dessous.

| Étape | Description |
|-------|-------------|
| 1. Analyse des modalités | Identifier les valeurs prises par chaque variable dans l'intervalle considéré. |
| 2. Analyse des valeurs manquantes | Évaluer la présence et la proportion de données manquantes pour chaque variable. |
| 3. Traitement des valeurs manquantes | Appliquer une méthode d’imputation ou de remplacement adaptée selon la nature des données. |
| 4. Transformation des variables | Encoder ou transformer les variables selon leur type et leur utilité dans les modèles. |

En vue d'analyser les valeurs manquantes associées à chaque variable deux méthodes sont créées :
1. `get_most_frequent_modality` qui affiche la distribution des modalités d'une variable d'un *DataFrame* en pourcentage et renvoie la modalité la plus fréquente ;
2. `impute_missing_values` qui remplace les valeurs manquantes d'une variable dans un *DataFrame*.

In [ ]:
def get_most_frequent_modality(df, variable, verbose=False):
    """
    Calcule et affiche la distribution des modalités d'une variable d'un
    DataFrame en pourcentage, et renvoie la modalité la plus fréquente.

    Paramètres :
    - df : DataFrame contenant la variable
    - variable : nom de la variable (colonne) à analyser
    - verbose : booléen, si True affiche les pourcentages

    Retour :
    - La modalité la plus fréquente (valeur)
    """
    distribution = (
        df[variable].value_counts(normalize=True, dropna=False) * 100
    )
    most_frequent_modality = distribution.idxmax()
    if verbose:
        print(f"Répartition des modalités pour la variable '{variable}' :")
        for modality, pct in distribution.items():
            print(f"  {repr(modality)} : {pct:.2f}%")
        print(
            f"Modalité la plus fréquente pour {variable} : "
            f"{most_frequent_modality}"
        )

    return most_frequent_modality

In [ ]:
def impute_missing_values(df, variable, fill_value):
    """
    Remplace les valeurs manquantes d'une variable dans un DataFrame.

    Paramètres :
    - df : DataFrame cible
    - variable : nom de la variable (colonne) à traiter (str)
    - fill_value : valeur utilisée pour remplacer les valeurs manquantes

    Retour :
    - DataFrame avec les valeurs manquantes remplacées pour la variable
      spécifiée
    """
    df[variable] = df[variable].fillna(fill_value)
    return df

### Variables disposant d'exactement 2 modalités

#### Analyse des modalités

In [ ]:
variables_0_2_str = extract_variables_from_interval(df_card_summary, "0-2")
unique_values_per_variable = get_unique_values_per_variable(
    X_train_clean, variables_0_2_str
)
display_scrollable_df(unique_values_per_variable)

Le tableau ci-dessous regroupe les variables disposant de deux modalités selon les valeurs qu’elles prennent, et propose une stratégie de transformation adaptée à chaque cas.

| Modalités uniques        | Variables                                                                                                                                                                          | Action recommandée                         |
| ------------------------ | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------ |
| `[0, 1]`                 | `EQUIPEMENT1`, `EQUIPEMENT4`, `KAPITAL1`, `KAPITAL3`, `KAPITAL6`, `KAPITAL7`, `KAPITAL8`, `KAPITAL9`                                                                               | Déjà binaire – pas de transformation       |
| `[N, O]`                 | `ADOSS`, `DEROG11`, `DEROG2`, `DEROG3`, `DEROG4`, `DEROG5`, `DEROG8`, `INDEM1`, `KAPITAL34`, `KAPITAL35`, `KAPITAL37`, `KAPITAL40`, `KAPITAL41`, `KAPITAL42`, `KAPITAL43`, `RISK8` | Mapping binaire (`N` $\rightarrow$ 0, `O` $\rightarrow$ 1)             |
| `[01. <= 10, 02. <= 20]` | `IND_0_Y1`, `IND_Y2_Y3`, `IND_Y3_Y4`, `IND_Y4_Y5`, `LOG_INC`                                                                                                                       | Encodage ordinal (`01.` $\rightarrow$ 1, `02.` $\rightarrow$ 2)       |
| `[1, 2]`                 | `TYPERS`                                                                                                                                                                           | Ordinal ou booléen selon sémantique métier |

Dans le cadre de ce projet, aucune information métier spécifique n’a été communiquée concernant la variable `TYPERS`. En l’absence de contexte complémentaire, les modalités existantes de cette variable ont été conservées en l’état pour les traitements à venir.

#### Variables déjà binaires

##### Analyse des valeurs manquantes

In [ ]:
# Filtrage des variables ayant exactement ['0', '1'] comme modalités
vars_0_1 = unique_values_per_variable[
    unique_values_per_variable["Modalités"].apply(
        lambda x: set(map(str, x)) == {"0", "1"}
    )
]["Variable"].tolist()

filtered_missing_info_vars_0_1 = missing_info_X_train[
    missing_info_X_train["Variable"].isin(vars_0_1)
]
display_scrollable_df(filtered_missing_info_vars_0_1)

Parmi les variables déjà sous forme binaire aucune ne dispose de valeur manquante. Aucune transformation n'est requise.

#### *Mapping* binaire (`N` $\rightarrow$ 0, `O` $\rightarrow$ 1)

Réalisons le *mapping* pour les variables : `ADOSS`, `DEROG11`, `DEROG2`, `DEROG3`, `DEROG4`, `DEROG5`, `DEROG8`, `INDEM1`, `KAPITAL34`, `KAPITAL35`, `KAPITAL37`, `KAPITAL40`, `KAPITAL41`, `KAPITAL42`, `KAPITAL43`, `RISK8`.

Filtrons d'abord les variables qui ont comme modalités [`N`, `O`] dans `unique_values_per_variable`

In [ ]:
# Filtrage des variables ayant exactement ['N', 'O'] comme modalités
vars_N_O = unique_values_per_variable[
    unique_values_per_variable["Modalités"].apply(
        lambda x: set(x) == {"N", "O"}
    )
]["Variable"].tolist()

##### Analyse des valeurs manquantes

Analysons à présent les variables de sorte à pouvoir traiter les valeurs manquantes potentielles.

| Variable    | Interprétation probable                                           | Suggestion sur NaN                                          |
| ----------- | ----------------------------------------------------------------- | ----------------------------------------------------------- |
| `ADOSS`     | Probablement : bien adossé à un mur (adossement)                  | NaN = non renseigné → peut-être "N" ou "inconnu"            |
| `DEROG11`   | Dérogation 11 (ex. urbanisme, fiscalité)                          | Peut être interprété comme **absence de dérogation** si NaN |
| `DEROG2`    | Idem – une autre forme de dérogation                              | Même logique que `DEROG11`                                  |
| `DEROG3`    | Voir ci-dessus                                                    | Idem                                                        |
| `DEROG4`    | Voir ci-dessus                                                    | Idem                                                        |
| `DEROG5`    | Voir ci-dessus                                                    | Idem                                                        |
| `DEROG8`    | Voir ci-dessus                                                    | Idem                                                        |
| `INDEM1`    | Peut faire référence à un critère d’indemnisation 1 (éligibilité) | NaN = non éligible ou non renseigné                         |
| `KAPITAL34` | Composante du capital (indemnité ou contrat 34)                   | NaN = non applicable ?                                      |
| `KAPITAL35` | Composante du capital (type 35)                                   | Idem                                                        |
| `KAPITAL37` | Idem                                                              | Idem                                                        |
| `KAPITAL40` | Idem                                                              | Idem                                                        |
| `KAPITAL41` | Idem                                                              | Idem                                                        |
| `KAPITAL42` | Idem                                                              | Idem                                                        |
| `KAPITAL43` | Idem                                                              | Idem                                                        |
| `RISK8`     | Un type de risque n°8 (incendie ? zone ? structure ?)             | NaN = risque non calculé / non concerné                     |

Avant de faire un choix de traitement sur les valeurs manquantes, affichons les variables concernées à partir du *datframe* `missing_info_X_train` généré en amont.

In [ ]:
missing_info_X_train_clean = summarize_missing_values(
    X_train_clean, threshold=0, sort=True
)
filtered_missing_info_vars_N_O = missing_info_X_train_clean[
    missing_info_X_train_clean["Variable"].isin(vars_N_O)
]
display_scrollable_df(filtered_missing_info_vars_N_O)

##### Traitement des valeurs manquantes

Puisque la variable `RISK8` est la seule parmi celles étudiées à contenir des valeurs manquantes, étudions de plus prés la distribution de ces modalités.

In [ ]:
# Calcul des fréquences
variable_str = "RISK8"
most_frequent_modality_RISK8 = get_most_frequent_modality(
    X_train_clean, variable_str, verbose=True
)

La distribution des modalités de la variable `RISK8` est la suivante :
- `'O'` : 62.84 % ;
- `'N'` : 30.23 % ;
- Valeurs manquantes (`NaN`) : 6.93 %

La modalité la plus fréquente est donc `'O'`.

Deux stratégies sont envisageables pour remplacer les valeurs manquantes :

1. Remplacement par la modalité majoritaire (`'O'`)

Cette option est statistiquement cohérente dans l'objectif de préserver la distribution observée de la variable. Elle est souvent utilisée lorsque les modalités sont considérées comme des états d'information équivalents ou interchangeables du point de vue du modèle.

2. Remplacement par `'N'` (absence du risque)

Cette alternative repose sur une hypothèse métier : si `RISK8` désigne un type de risque et que sa valeur est manquante, cela pourrait indiquer que le risque n’a pas été détecté ou calculé, et donc présumé absent. Cette interprétation est valide si l’absence de valeur reflète effectivement un non-événement plutôt qu’un oubli ou une erreur de saisie.


Le choix entre `'O'` et `'N'` dépendra du sens métier de la variable :
- Si une valeur manquante implique l'absence du risque, alors il est pertinent de la remplacer par `'N'` ;
- Si les valeurs manquantes sont considérées comme aléatoires ou indépendantes du risque, alors un remplacement par `'O'` permet de préserver la distribution globale ;
En l’absence d’information complémentaire sur la provenance des valeurs manquantes, les deux hypothèses seront utilisées dans un cadre expérimental (*cross-validation* ou *validation externe*) pour mesurer leur impact sur la performance du modèle.

Avant de procéder à l'imputation, vérifions si l'hypothèse métier est cohérente.

Dans le cadre de l'objectif général — modéliser la charge sinistre incendie (CHARGE) — trois composantes sont à disposition :

- `FREQ` (fréquence des sinistres) ;
- `CM` (coût moyen des sinistres)
- `ANNEE_ASSURANCE` (durée d’exposition)
- `CHARGE` = `FREQ` $\times$ `CM` $\times$ `ANNEE_ASSURANCE`.

Pour évaluer si une valeur manquante de `RISK8` doit être imputée par `'N'` (pas de risque) ou `'O'` (présence de risque), la variable à privilégier est `FREQ` et ce pour deux raisons :
- Si `FREQ` est nulle ou très faible pour les lignes où `RISK8` est `NaN` ou `'N'`, cela indique une absence ou rareté de sinistres ;
- Cela confirme que l'absence de données dans `RISK8` est liée à un profil peu exposé au risque, ce qui justifie l’imputation par `'N'`.

Ainsi un filtrage sur les lignes avec `RISK8` manquant ou `'N'` est réalisé de sorte à comparer la fréquence moyenne.

Deux variables sont définies `mean_freq_no_risk` et `mean_freq_overall`, respectivement associées à la moyenne de fréquence lorsque `RISK8` prend pour modalité `NaN` ou `'N'` et la moyenne de fréquence globale.

Si `mean_freq_no_risk` $\ll$ `mean_freq_overall`, cela renforce l'hypothèse métier et les valeurs manquantes pour `RISK8` doivent être remplacées par `'N'`

In [ ]:
mask = X_train_clean["RISK8"].isin(["N", np.nan])
mean_freq_no_risk = y_train.loc[mask, "FREQ"].mean()
mean_freq_overall = y_train["FREQ"].mean()

print(f"Fréquence moyenne (RISK8=N ou NaN) : {mean_freq_no_risk:.4f}")
print(f"Fréquence moyenne globale          : {mean_freq_overall:.4f}")

In [ ]:
y_train["FREQ"].loc[X_train_clean["RISK8"].isna()].hist(
    alpha=0.9, label="RISK8 = NaN"
)
y_train["FREQ"].loc[X_train_clean["RISK8"] == "N"].hist(
    alpha=0.5, label="RISK8 = N"
)
y_train["FREQ"].loc[X_train_clean["RISK8"] == "O"].hist(
    alpha=0.5, label="RISK8 = O"
)
plt.legend()
plt.title("Distribution de FREQ selon les modalités de RISK8")
plt.xlabel("FREQ")
plt.ylabel("Nombre d'observations")
plt.show()

**Analyse du résultat :**
- Fréquence moyenne (`RISK8` = `'N'` ou `NaN`) : `0.0136` ;
- Fréquence moyenne globale : `0.0125`

**Cela indique que ces profils ont en moyenne une fréquence de sinistres un peu plus élevée que l’ensemble.**

Ainsi :
- `RISK8` n’est peut-être pas fiable : Si elle contient des `NaN` pour des profils qui ont bel et bien des sinistres, c’est que l'absence de valeur n’indique pas une absence de risque. **Donc imputer par `'N'` serait risqué, car cela pourrait masquer un effet réel.**
- Le champ `RISK8` n’est pas corrélé à `FREQ`, ou pas de façon intuitive. Une valeur `'O'` n'implique pas nécessairement un sinistre plus probable.

**Par conséquent, l'imputation automatique des valeurs manquantes de `RISK8` par `'N'` n'est pas justifiée.**

Deux options restent préférables :
- Imputation par la modalité majoritaire (`'O'`), ce qui constitue une stratégie neutre dans un cadre prédictif. Cependant, si la variable RISK8 est effectivement indicative d’un risque, une imputation prudente (par 'O') s’impose afin de ne pas sous-estimer ce dernier.
- Créer une modalité `'MISSING'` ou `'_NA'`, pour conserver l'information de manque. Cela peut être informatif pour un modèle.

La stratégie d’imputation appliquée ici (ex. : remplacement de RISK8 par `'O'`) sera amenée à évoluer en fonction des performances des modèles construits ultérieurement.
En effet, si le modèle obtient de meilleures performances en imputant par la modalité opposée (ex. `'N'` au lieu de `'O'`), ou en excluant certaines variables, il sera pertinent de revenir sur cette étape et d’ajuster l’imputation en conséquence.
C'est la raison pour laquelle cette cellule est volontairement isolée pour faciliter sa modification après évaluation des modèles

In [ ]:
# Cette cellule impute les valeurs manquantes de la variable RISK8.
# Elle est volontairement isolée afin de pouvoir facilement tester d'autres
# stratégies selon les performances des modèles (ex. imputation par 'O' ou
# ou 'MISSING').

# Version A : imputation par la modalité la plus fréquente
impute_missing_values(
    X_train_clean, "RISK8", fill_value=most_frequent_modality_RISK8
);

# Version B : création d'une modalité MISSING
# X_train_clean["RISK8"] = X_train_clean["RISK8"].fillna("MISSING")

Vérifions que `RISK8` ne dispose plus de valeurs manquantes.

In [ ]:
missing_info_X_train_clean = summarize_missing_values(
    X_train_clean, threshold=0, sort=True
)
filtered_missing_info_vars_N_O = missing_info_X_train_clean[
    missing_info_X_train_clean["Variable"].isin(vars_N_O)
]
display_scrollable_df(filtered_missing_info_vars_N_O)

##### Transformation des variables

Effectuons ensuite le *mapping* binaire.

In [ ]:
# Transformation des variables avec modalités ['N', 'O'] en variables binaires
for var in vars_N_O:
    X_train_clean[var] = X_train_clean[var].map({"N": 0, "O": 1})

In [ ]:
X_train_clean[vars_N_O].head()

Les variables ont bien été converties et aucune ne dispose de valeurs manquantes.

#### Encodage ordinal (`01.` $\rightarrow$ 1, `02.` $\rightarrow$ 2)

Dans la démarche analogue à celle réalisée précédemment, filtrons d'abord les variables qui ont comme modalités [`01. <= 10`, `02. <= 20`] dans `unique_values_per_variable`

In [ ]:
# Identification des variables avec des modalités ordinales spécifiques
modalites_ordinales = {"01. <= 10", "02. <= 20"}
vars_ordinal = unique_values_per_variable[
    unique_values_per_variable["Modalités"].apply(
        lambda x: set(x) == modalites_ordinales
    )
]["Variable"].tolist()

##### Analyse des valeurs manquantes

Analysons préalablement les variables de sorte à pouvoir traiter les valeurs manquantes potentielles.

###### `IND_0_Y1`, `IND_Y2_Y3`, `IND_Y3_Y4`, `IND_Y4_Y5`.
Ces variables suivent un motif cohérent :
- `IND` pourrait désigner "indice", "indicateur" ou "individualisation" ;
- Les suffixes font penser à des intervalles temporels, probablement des périodes glissantes d’un an :
    - `0_Y1` : de l'année 0 à l'année 1 (1ère année)
    - `Y2_Y3` : de la 2e à la 3e année ;
    - etc.

Elles semblent donc représenter une mesure binaire ou ordinale sur une période donnée, comme :

Une indication de changement de statut (ex. : évolution du risque, du revenu, de la sinistralité) ;

Un score discret mesurant la variation d’une situation sur une année.

Le fait que leurs modalités soient `['01. <= 10', '02. <= 20']` pourrait indiquer une quantification regroupée par tranche :
- `01. <= 10` : score ou valeur faible ;
- `02. <= 20` : score ou valeur modérée.

###### `LOG_INC`

Ce nom évoque fortement :
- `LOG` : logement (fréquent en bases de données sur l’habitat) ou logarithme (moins probable ici)
- `INC` : incendie ou incidents.

Puisque la variable est binaire (`['01. <= 10', '02. <= 20']`), elle pourrait représenter :
- Une typologie de logement avec deux classes ;
- Un indicateur de niveau de précarité ou de conformité ;
- Ou encore une mesure d’incident ou sinistre dans un logement, segmentée par intensité.

| Variable    | Interprétation probable                                                  |
| ----------- | ------------------------------------------------------------------------ |
| `IND_0_Y1`  | Indicateur pour la 1re année (binaire ou ordinal, score discret)         |
| `IND_Y2_Y3` | Idem pour années 2 à 3                                                   |
| `IND_Y3_Y4` | Idem pour années 3 à 4                                                   |
| `IND_Y4_Y5` | Idem pour années 4 à 5                                                   |
| `LOG_INC`   | Logement concerné par incident / inconfort / indicateur ordinal logement |

In [ ]:
missing_info_X_train_clean = summarize_missing_values(
    X_train_clean, threshold=0, sort=True
)
filtered_missing_info_vars_ordinal = missing_info_X_train_clean[
    missing_info_X_train_clean["Variable"].isin(vars_ordinal)
]
display_scrollable_df(filtered_missing_info_vars_ordinal)

Chacune des variables `IND_0_Y1`, `IND_Y2_Y3`, `IND_Y3_Y4`, `IND_Y4_Y5` et `LOG_INC` disposent de 4,85% de valeurs manquantes

##### Traitement des valeurs manquantes

Étapes à suivre pour chaque variable :
- Créer un sous-groupe par modalité (`'01. <= 10'`, `'02. <= 20'`) ;
- Calculer la fréquence moyenne FREQ par modalité ;
- Comparer ces moyennes ;
- Visualiser les différences.

Pour cela est implémentée `analyze_freq_by_modality` qui affiche la fréquence moyenne (FREQ) pour chaque modalité d'une variable donnée.

- Si la fréquence moyenne varie nettement entre les deux modalités, cela confirme l’intérêt du codage ordinal (ex: 1 pour `'01. <= 10'`, 2 pour `'02. <= 20'`) ;
- Si les fréquences sont quasi identiques, le codage peut être simplifié ou la variable considérée comme peu informative.

| Variable    | Modalités                    | Traitement recommandé des valeurs manquantes                  | Justification                                                 |
| ----------- | ---------------------------- | ------------------------------------------------------------- | ------------------------------------------------------------- |
| `IND_0_Y1`  | `'01. <= 10'`, `'02. <= 20'` | Imputation par la **modalité la plus faible** : `'01. <= 10'` | Hypothèse prudente : valeur manquante = pas d’indicateur fort |
| `IND_Y2_Y3` | Idem                         | Idem                                                          | Idem                                                          |
| `IND_Y3_Y4` | Idem                         | Idem                                                          | Idem                                                          |
| `IND_Y4_Y5` | Idem                         | Idem                                                          | Idem                                                          |
| `LOG_INC`   | Idem                         | Idem                                                          | Idem                                                          |

In [ ]:
def analyze_freq_by_modality(X, y, variable, target="FREQ"):
    """
    Analyse la moyenne de la variable cible (ex: FREQ) selon les modalités
    d'une variable explicative.

    Paramètres :
    - X : DataFrame des variables explicatives
    - y : DataFrame ou Series contenant la variable cible
    - variable : nom de la variable à analyser
    - target : nom de la variable cible (par défaut 'FREQ')
    """
    df = X[[variable]].copy()
    df[target] = y[target]

    result = df.groupby(variable)[target].mean().sort_index()
    print(f"Fréquence moyenne selon les modalités de '{variable}' :")
    for modality, val in result.items():
        print(f"  {modality} : {val:.4f}")
    return result

In [ ]:
variables_to_analyze = [
    "IND_0_Y1",
    "IND_Y2_Y3",
    "IND_Y3_Y4",
    "IND_Y4_Y5",
    "LOG_INC",
]

for var in variables_to_analyze:
    analyze_freq_by_modality(X_train_clean, y_train, var)
    print("-" * 50)

Le tableau ci-dessous résume les résultats et interprétations des variables `IND_0_Y1`, `IND_Y2_Y3`, `IND_Y3_Y4`, `IND_Y4_Y5`, `LOG_INC` selon la variable `FREQ`.

| Variable     | Modalité         | FREQ moyenne | Interprétation                                                                 |
|--------------|------------------|--------------|---------------------------------------------------------------------------------|
| IND_0_Y1     | `01. <= 10`      | 0.0124       | Fréquence normale                                                               |
|              | `02. <= 20`      | 0.0000       | Aucun sinistre : modalité très faible risque                                   |
| IND_Y2_Y3    | `01. <= 10`      | 0.0124       | Fréquence normale                                                               |
|              | `02. <= 20`      | 0.0147       | Plus risquée : tendance opposée à IND_0_Y1                                     |
| IND_Y3_Y4    | `01. <= 10`      | 0.0116       | Fréquence légèrement inférieure                                                 |
|              | `02. <= 20`      | 0.0169       | Modalité plus exposée aux sinistres                                            |
| IND_Y4_Y5    | `01. <= 10`      | 0.0125       | Fréquence normale                                                               |
|              | `02. <= 20`      | 0.0054       | Moins risquée : population plus stable ?                                       |
| LOG_INC      | `01. <= 10`      | 0.0124       | Fréquence normale                                                               |
|              | `02. <= 20`      | 0.0000       | Aucun sinistre : peut indiquer un logement inactif ou à très faible risque     |

Ce tableau met en évidence l'absence de tendance univoque entre les variables, ce qui appelle à la prudence dans le choix de l’imputation des valeurs manquantes : soit neutre (modalité majoritaire), soit prudente (modalité plus risquée) selon le contexte métier ou les performances en validation.

Étant donné que l’objectif est d’estimer une prime incendie, et par conséquent d’évaluer le risque incendie associé, il est préférable d’adopter une approche prudente. Dans ce contexte, cela signifie imputer les valeurs manquantes avec la modalité la plus exposée au risque pour chaque variable.

Les imputations retenues sont donc :
- `IND_0_Y1` : `01. <= 10` ;
- `IND_Y2_Y3` : `02. <= 20` ;
- `IND_Y3_Y4` : `02. <= 20` ;
- `IND_Y4_Y5` : `01. <= 10` ;
- `LOG_INC` : `01. <= 10`.

Cette stratégie permet de ne pas sous-estimer le risque potentiel, ce qui est cohérent avec une logique assurantielle préventive.

In [ ]:
# Valeur de remplacement choisie pour les modalités manquantes
replacement_values = {
    "IND_0_Y1": "01. <= 10",
    "IND_Y2_Y3": "02. <= 20",
    "IND_Y3_Y4": "02. <= 20",
    "IND_Y4_Y5": "01. <= 10",
    "LOG_INC": "01. <= 10",
}

for var in vars_ordinal:
    replacement_value = replacement_values.get(var)
    X_train_clean[var] = X_train_clean[var].fillna(replacement_value)

Vérifions qu'il ne subsiste plus de valeurs manquantes.

In [ ]:
missing_info_X_train_clean = summarize_missing_values(
    X_train_clean, threshold=0, sort=True
)
filtered_missing_info_vars_ordinal = missing_info_X_train_clean[
    missing_info_X_train_clean["Variable"].isin(vars_ordinal)
]
display_scrollable_df(filtered_missing_info_vars_ordinal)

Effectuons ensuite l'encodage ordinal.

In [ ]:
# Transformation des variables avec modalités ['01. <= 10', '02. <= 20'] en
# variables ordinales
ordinal_mapping = {"01. <= 10": 1, "02. <= 20": 2}
for var in vars_ordinal:
    X_train_clean[var] = X_train_clean[var].map(ordinal_mapping)

Vérifions le résultat.

In [ ]:
X_train_clean[vars_ordinal].head()

#### Variable `TYPERS`

##### Analyse des valeurs manquantes

In [ ]:
# Filtrage des variables ayant exactement ['0', '1'] comme modalités
vars_typers = unique_values_per_variable[
    unique_values_per_variable["Modalités"].apply(
        lambda x: set(map(str, x)) == {"1", "2"}
    )
]["Variable"].tolist()

filtered_missing_info_vars_typers = missing_info_X_train[
    missing_info_X_train["Variable"].isin(vars_typers)
]
display_scrollable_df(filtered_missing_info_vars_typers)

Il n'y a pas de valeurs manquantes pour la variable `TYPERS`, aucun traitement n'est requis.

#### Conclusion quant au traitement des variables disposant de deux modalités

| Variables | Modalités uniques | Action | Variables traitées pour valeurs manquantes | Traitement réalisé sur les valeurs manquantes |
| --------- | ----------------- | ------ | ------------------------------------------ | ---------------------------------------------- |
| `EQUIPEMENT1`, `EQUIPEMENT4`, `KAPITAL1`, `KAPITAL3`, `KAPITAL6`, `KAPITAL7`, `KAPITAL8`, `KAPITAL9` | `[0, 1]` | Déjà binaire – pas de transformation | ∅ | Aucun |
| `ADOSS`, `DEROG11`, `DEROG2`, `DEROG3`, `DEROG4`, `DEROG5`, `DEROG8`, `INDEM1`, `KAPITAL34`, `KAPITAL35`, `KAPITAL37`, `KAPITAL40`, `KAPITAL41`, `KAPITAL42`, `KAPITAL43`, `RISK8` | `[N, O]` | Mapping binaire (`N` $\rightarrow$ 0, `O` $\rightarrow$ 1) | `RISK8` | `O` |
| `IND_0_Y1`, `IND_Y2_Y3`, `IND_Y3_Y4`, `IND_Y4_Y5`, `LOG_INC` | `[01. <= 10, 02. <= 20]` | Encodage ordinal (`01.` $\rightarrow$ 1, `02.` $\rightarrow$ 2) | `IND_0_Y1` $\rightarrow$ `01. <= 10`  <br> `IND_Y2_Y3` $\rightarrow$ `02. <= 20`  <br> `IND_Y3_Y4` $\rightarrow$ `02. <= 20`  <br> `IND_Y4_Y5` $\rightarrow$ `01. <= 10`  <br> `LOG_INC` $\rightarrow$ `01. <= 10` | Imputation par la modalité la plus risquée selon `FREQ` |
| `TYPERS` | `[1, 2]` | En l’absence de contexte complémentaire, modalités conservées en l’état | ∅ | Aucun |

### Variables disposant de 3 à 5 modalités

#### Analyse des modalités

In [ ]:
variables_3_5_str = extract_variables_from_interval(df_card_summary, "3-5")
unique_values_per_variable = get_unique_values_per_variable(
    X_train_clean, variables_3_5_str
)
unique_values_per_variable = unique_values_per_variable.sort_values(
    by=["Typologie détectée", "Variable"]
)
display_scrollable_df(unique_values_per_variable)

Le tableau ci-dessus offre une vue d'ensemble des variables, des modalités associées et de leur typologie.
Cependant, certaines variables partagent exactement les mêmes modalités. Par exemple, les variables `CARACT1`, `EQUIPEMENT2`, `RISK10`, `RISK11`, `RISK12`, `RISK13`, `RISK9` ont toutes pour modalités `[N,O,R]`.

Afin de mieux visualiser ces redondances et de simplifier l’analyse, regroupons désormais les variables en fonction de leurs modalités communes.

Pour cela, nous allons utiliser la méthode `fusionner_variables_par_modalites` qui permet de fusionner les variables qui partagent exactement les **mêmes modalités** dans un *DataFrame*, tout en conservant les informations suivantes :
- Les **modalités associées**
- Le **nombre de modalités**
- La **typologie détectée**
- Un **tri alphabétique** selon le premier nom de variable de chaque groupe

In [ ]:
def fusionner_variables_par_modalites(df):
    """
    Fusionne les variables d'un DataFrame qui partagent exactement les mêmes
    modalités.

    Paramètres :
        df (pd.DataFrame) : Doit contenir les colonnes suivantes :
            - 'Variable'
            - 'Modalités'
            - 'Nombre de Modalités'
            - 'Typologie détectée'

    Retour :
        pd.DataFrame : DataFrame fusionné où les variables ayant les mêmes
        modalités sont regroupées.
    """

    # Étape 1 : Créer une version textuelle triée des modalités
    df = df.copy()
    df["Modalites_str"] = df["Modalités"].apply(
        lambda x: str(sorted(x)) if isinstance(x, list) else str(x)
    )

    # Étape 2 : Grouper par cette chaîne
    grouped = df.groupby("Modalites_str")

    # Étape 3 : Identifier les groupes avec des modalités partagées
    duplicated_groups = grouped.filter(lambda x: len(x) > 1)

    # Étape 4 : Regrouper les variables de modalités communes
    grouped_duplicates = (
        duplicated_groups.groupby("Modalites_str")
        .agg(
            {
                "Variable": list,
                "Modalités": "first",
                "Nombre de Modalités": "first",
                "Typologie détectée": "first",
            }
        )
        .reset_index(drop=True)
    )

    # Étape 5 : Repérer les variables à modalités uniques
    unique_modalites = df[
        ~df["Modalites_str"].isin(duplicated_groups["Modalites_str"])
    ]

    # Étape 6 : Uniformiser les formats
    unique_modalites = unique_modalites.copy()
    unique_modalites["Variable"] = unique_modalites["Variable"].apply(
        lambda x: [x]
    )

    # Étape 7 : Fusion finale
    final_df = pd.concat(
        [
            grouped_duplicates,
            unique_modalites[
                [
                    "Variable",
                    "Modalités",
                    "Nombre de Modalités",
                    "Typologie détectée",
                ]
            ],
        ],
        ignore_index=True,
    )

    # Ajouter une colonne temporaire pour le tri
    final_df["Variable_tri"] = final_df["Variable"].apply(lambda x: x[0])

    # Trier selon le premier nom de variable
    final_df = (
        final_df.sort_values(by="Variable_tri")
        .drop(columns="Variable_tri")
        .reset_index(drop=True)
    )

    return final_df

In [ ]:
test = fusionner_variables_par_modalites(unique_values_per_variable)
display_scrollable_df(test)

Le tableau suivant regroupe les variables en fonction de leurs modalités communes.

| Variable                                                                        | Modalités                                                              | Nombre de Modalités | Typologie détectée   |
|---------------------------------------------------------------------------------|------------------------------------------------------------------------|---------------------|----------------------|
| [`ALTITUDE_1`]                                                                    | [01. &lt;= 190, 02. &lt;= 438, 03. &lt;= 794, 04. &gt;= 794]           | 4                   | Continue discrétisée |
| [`ALTITUDE_2`]                                                                    | [01. &lt;= 239, 02. &lt;= 588, 03. &lt;= 1186, 04. &gt;= 1186]         | 4                   | Continue discrétisée |
| [`ALTITUDE_3`]                                                                    | [01. &lt;= 236, 02. &lt;= 579, 03. &lt;= 1178, 04. &gt;= 1178]         | 4                   | Continue discrétisée |
| [`ALTITUDE_4`]                                                                    | [01. &lt;= 143, 02. &lt;= 333, 03. &lt;= 645, 04. &gt;= 645]           | 4                   | Continue discrétisée |
| [`ALTITUDE_5`]                                                                    | [01. &lt;= 337, 02. &lt;= 840, 03. &lt;= 1810, 04. &gt;= 1810]         | 4                   | Continue discrétisée |
| [`BDTOPO_BAT_MAX_HAUTEUR`]                                                        | [01. &lt;= 5, 02. &lt;= 6, 03. &lt;= 9, 04. &gt;= 9]                   | 4                   | Continue discrétisée |
| [`BDTOPO_BAT_MAX_HAUTEUR_MAX`]                                                    | [01. &lt;= 6, 02. &lt;= 9, 03. &lt;= 13, 04. &gt;= 13]                 | 4                   | Continue discrétisée |
| [`CA3`]                                                                           | [0, 1000, 6000, 30000, 50000]                                          | 5                   | Ordinale             |
| [`CARACT1`, `EQUIPEMENT2`, `RISK10`, `RISK11`, `RISK12`, `RISK13`, `RISK9`]                   | [N, O, R]                                                              | 3                   | Nominale             |
| [`DISTANCE_1`]                                                                    | [01. &lt;= 3, 02. &lt;= 6, 03. &lt;= 940, 04. &gt;= 940]               | 4                   | Continue discrétisée |
| [`DISTANCE_111`]                                                                  | [01. &lt;= 13, 02. &lt;= 25, 03. &lt;= 961, 04. &gt;= 961]             | 4                   | Continue discrétisée |
| [`DISTANCE_112`]                                                                  | [01. &lt;= 1, 02. &lt;= 4, 03. &lt;= 937, 04. &gt;= 937]               | 4                   | Continue discrétisée |
| [`DISTANCE_121`]                                                                  | [01. &lt;= 6, 02. &lt;= 13, 03. &lt;= 952, 04. &gt;= 952]              | 4                   | Continue discrétisée |
| [`DISTANCE_122`]                                                                  | [01. &lt;= 13, 02. &lt;= 28, 03. &lt;= 959, 04. &gt;= 959]             | 4                   | Continue discrétisée |
| [`DISTANCE_123`]                                                                  | [01. &lt;= 48, 02. &lt;= 92, 03. &lt;= 145, 04. &gt;= 145]             | 4                   | Continue discrétisée |
| [`DISTANCE_124`]                                                                  | [01. &lt;= 16, 02. &lt;= 30, 03. &lt;= 976, 04. &gt;= 976]             | 4                   | Continue discrétisée |
| [`DISTANCE_131`]                                                                  | [01. &lt;= 11, 02. &lt;= 22, 03. &lt;= 952, 04. &gt;= 952]             | 4                   | Continue discrétisée |
| [`DISTANCE_132`]                                                                  | [01. &lt;= 23, 02. &lt;= 42, 03. &lt;= 67, 04. &gt;= 67]               | 4                   | Continue discrétisée |
| [`DISTANCE_133`]                                                                  | [01. &lt;= 40, 02. &lt;= 77, 03. &lt;= 136, 04. &gt;= 136]             | 4                   | Continue discrétisée |
| [`DISTANCE_141`]                                                                  | [01. &lt;= 20, 02. &lt;= 36, 03. &lt;= 58, 04. &gt;= 58]               | 4                   | Continue discrétisée |
| [`DISTANCE_142`]                                                                  | [01. &lt;= 8, 02. &lt;= 17, 03. &lt;= 946, 04. &gt;= 946]              | 4                   | Continue discrétisée |
| [`DISTANCE_2`]                                                                    | [01. &lt;= 2, 02. &lt;= 10, 03. &gt;= 10]                              | 3                   | Continue discrétisée |
| [`DISTANCE_211`]                                                                  | [01. &lt;= 4, 02. &lt;= 14, 03. &lt;= 945, 04. &gt;= 945]              | 4                   | Continue discrétisée |
| [`DISTANCE_212`]                                                                  | [01. &lt;= 270, 02. &lt;= 467, 03. &lt;= 666, 04. &gt;= 666]           | 4                   | Continue discrétisée |
| [`DISTANCE_213`]                                                                  | [01. &lt;= 236, 02. &lt;= 426, 03. &lt;= 614, 04. &gt;= 614]           | 4                   | Continue discrétisée |
| [`DISTANCE_221`]                                                                  | [01. &lt;= 31, 02. &lt;= 83, 03. &lt;= 152, 04. &gt;= 152]             | 4                   | Continue discrétisée |
| [`DISTANCE_222`]                                                                  | [01. &lt;= 19, 02. &lt;= 45, 03. &lt;= 101, 04. &gt;= 101]             | 4                   | Continue discrétisée |
| [`DISTANCE_223`]                                                                  | [01. &lt;= 191, 02. &lt;= 382, 03. &lt;= 568, 04. &gt;= 568]           | 4                   | Continue discrétisée |
| [`DISTANCE_231`]                                                                  | [01. &lt;= 2, 02. &lt;= 7, 03. &lt;= 939, 04. &gt;= 939]               | 4                   | Continue discrétisée |
| [`DISTANCE_242`]                                                                  | [01. &lt;= 1, 02. &lt;= 5, 03. &lt;= 938, 04. &gt;= 938]               | 4                   | Continue discrétisée |
| [`DISTANCE_243`]                                                                  | [01. &lt;= 2, 02. &lt;= 5, 03. &lt;= 937, 04. &gt;= 937]               | 4                   | Continue discrétisée |
| [`DISTANCE_311`, `DISTANCE_312`, `DISTANCE_313`, `DISTANCE_324`]                        | [01. &lt;= 10, 02. &lt;= 25, 03. &lt;= 50, 04. &lt;=100, 05. &gt;=100] | 5                   | Continue discrétisée |
| [`DISTANCE_321`]                                                                  | [01. &lt;= 23, 02. &lt;= 52, 03. &lt;= 89, 04. &gt;= 89]               | 4                   | Continue discrétisée |
| [`DISTANCE_322`]                                                                  | [01. &lt;= 14, 02. &lt;= 30, 03. &lt;= 951, 04. &gt;= 951]             | 4                   | Continue discrétisée |
| [`DISTANCE_323`]                                                                  | [01. &lt;= 126, 02. &lt;= 287, 03. &lt;= 463, 04. &gt;= 463]           | 4                   | Continue discrétisée |
| [`DISTANCE_331`]                                                                  | [01. &lt;= 52, 02. &lt;= 111, 03. &lt;= 188, 04. &gt;= 188]            | 4                   | Continue discrétisée |
| [`DISTANCE_332`]                                                                  | [01. &lt;= 60, 02. &lt;= 126, 03. &lt;= 201, 04. &gt;= 201]            | 4                   | Continue discrétisée |
| [`DISTANCE_333`]                                                                  | [01. &lt;= 78, 02. &lt;= 183, 03. &lt;= 306, 04. &gt;= 306]            | 4                   | Continue discrétisée |
| [`DISTANCE_334`]                                                                  | [01. &lt;= 189, 02. &lt;= 354, 03. &lt;= 532, 04. &gt;= 532]           | 4                   | Continue discrétisée |
| [`DISTANCE_335`]                                                                  | [01. &lt;= 174, 02. &lt;= 332, 03. &lt;= 503, 04. &gt;= 503]           | 4                   | Continue discrétisée |
| [`DISTANCE_411`]                                                                  | [01. &lt;= 25, 02. &lt;= 56, 03. &lt;= 981, 04. &gt;= 981]             | 4                   | Continue discrétisée |
| [`DISTANCE_412`]                                                                  | [01. &lt;= 60, 02. &lt;= 114, 03. &lt;= 186, 04. &gt;= 186]            | 4                   | Continue discrétisée |
| [`DISTANCE_421`]                                                                  | [01. &lt;= 81, 02. &lt;= 173, 03. &lt;= 292, 04. &gt;= 292]            | 4                   | Continue discrétisée |
| [`DISTANCE_422`]                                                                  | [01. &lt;= 130, 02. &lt;= 242, 03. &lt;= 394, 04. &gt;= 394]           | 4                   | Continue discrétisée |
| [`DISTANCE_423`]                                                                  | [01. &lt;= 109, 02. &lt;= 240, 03. &lt;= 387, 04. &gt;= 387]           | 4                   | Continue discrétisée |
| [`DISTANCE_511`]                                                                  | [01. &lt;= 18, 02. &lt;= 38, 03. &lt;= 62, 04. &gt;= 62]               | 4                   | Continue discrétisée |
| [`DISTANCE_512`]                                                                  | [01. &lt;= 8, 02. &lt;= 17, 03. &lt;= 954, 04. &gt;= 954]              | 4                   | Continue discrétisée |
| [`DISTANCE_521`]                                                                  | [01. &lt;= 100, 02. &lt;= 185, 03. &lt;= 300, 04. &gt;= 300]           | 4                   | Continue discrétisée |
| [`DISTANCE_522`]                                                                  | [01. &lt;= 118, 02. &lt;= 249, 03. &lt;= 393, 04. &gt;= 393]           | 4                   | Continue discrétisée |
| [`DISTANCE_523`]                                                                  | [01. &lt;= 118, 02. &lt;= 250, 03. &lt;= 401, 04. &gt;= 401]           | 4                   | Continue discrétisée |
| [`DUREE_REQANEUF`]                                                                | [0.0, 2.0, 5.0, 10.0]                                                  | 4                   | Ordinale             |
| [`ESPINSEE`]                                                                      | [ESP1, ESP2, ESP3, ESP4]                                               | 4                   | Nominale             |
| [`FFM_VOR_COM_MMAX_A_Y`, `FFM_VOR_MMAX_A`]                                          | [01. &lt;= 2, 02. &lt;= 4, 03. &lt;= 5, 04. &gt;= 5]                   | 4                   | Continue discrétisée |
| [`FFM_VOR_COM_MM_A_Y`, `FFM_VOR_MM_A`]                                              | [01. &lt;= 1, 02. &lt;= 3, 03. &lt;= 4, 04. &gt;= 4]                   | 4                   | Continue discrétisée |
| [`FXI3SAB_VOR_COM_MMAX_A_Y`]                                                      | [01. &lt;= 5, 02. &lt;= 17, 03. &lt;= 30, 04. &gt;= 30]                | 4                   | Continue discrétisée |
| [`FXI3SAB_VOR_COM_MM_A_Y`]                                                        | [01. &lt;= 4, 02. &lt;= 12, 03. &lt;= 23, 04. &gt;= 23]                | 4                   | Continue discrétisée |
| [`FXI3SAB_VOR_MMAX_A`]                                                            | [01. &lt;= 11, 02. &lt;= 24, 03. &lt;= 32, 04. &gt;= 32]               | 4                   | Continue discrétisée |
| [`FXI3SAB_VOR_MM_A`]                                                              | [01. &lt;= 8, 02. &lt;= 17, 03. &lt;= 24, 04. &gt;= 24]                | 4                   | Continue discrétisée |
| [`FXIAB_VOR_MMAX_A`]                                                              | [01. &lt;= 12, 02. &lt;= 26, 03. &lt;= 29, 04. &gt;= 29]               | 4                   | Continue discrétisée |
| [`FXIAB_VOR_MM_A`, `TAMPLIAB_VOR_MMAX_A`, `TXMIN_VOR_MMAX_A`]                         | [01. &lt;= 9, 02. &lt;= 19, 03. &lt;= 22, 04. &gt;= 22]                | 4                   | Continue discrétisée |
| [`FXYAB_VOR_MMAX_A`]                                                              | [01. &lt;= 6, 02. &lt;= 14, 03. &lt;= 17, 04. &gt;= 17]                | 4                   | Continue discrétisée |
| [`FXYAB_VOR_MM_A`]                                                                | [01. &lt;= 5, 02. &lt;= 10, 03. &lt;= 12, 04. &gt;= 12]                | 4                   | Continue discrétisée |
| [`HAUTEUR`]                                                                       | [01. &lt;= 4, 02. &lt;= 6, 03. &lt;= 7, 04. &gt;= 7]                   | 4                   | Continue discrétisée |
| [`HAUTEUR_MAX`, `NBJFXY8_MM_A`]                                                     | [01. &lt;= 6, 02. &lt;= 8, 03. &lt;= 11, 04. &gt;= 11]                 | 4                   | Continue discrétisée |
| [`IND`]                                                                           | [01. &lt;= 33995, 02. &lt;= 318560, 03. &lt;= 1340814]                 | 3                   | Continue discrétisée |
| [`IND_SNV`]                                                                       | [01. &lt;= 21873, 02. &lt;= 24733, 03. &lt;= 29681, 04. &gt;= 29681]   | 4                   | Continue discrétisée |
| [`IND_Y5_Y6`, `IND_Y6_Y7`, `PROPORTION_14`]                                           | [01. &lt;= 10, 02. &lt;= 20, 03. &lt;= 30, 04. &lt;= 40]               | 4                   | Continue discrétisée |
| [`IND_Y7_Y8`, `IND_Y9`, `MEN_5IND`, `MEN_FMP`]                                          | [01. &lt;= 10, 02. &lt;= 20, 03. &lt;= 30]                             | 3                   | Continue discrétisée |
| [`KAPITAL13`]                                                                     | [0, 5000, 10000]                                                       | 3                   | Ordinale             |
| [`KAPITAL14`]                                                                     | [0, 5000, 30000, 50000]                                                | 4                   | Ordinale             |
| [`KAPITAL26`]                                                                     | [0, 5000, 30000, 50000, 100000]                                        | 5                   | Ordinale             |
| [`KAPITAL27`, `KAPITAL28`]                                                          | [0, 25000, 50000]                                                      | 3                   | Ordinale             |
| [`KAPITAL29`, `KAPITAL30`, `KAPITAL31`]                                               | [0.0, 25000.0, 50000.0]                                                | 3                   | Ordinale             |
| [`MEN`]                                                                           | [01. &lt;= 17204, 02. &lt;= 153098, 03. &lt;= 670263]                  | 3                   | Continue discrétisée |
| [`MEN_SURF`]                                                                      | [01. &lt;= 93, 02. &lt;= 106, 03. &lt;= 118, 04. &gt;= 118]            | 4                   | Continue discrétisée |
| [`NBJFF10_MMAX_A`]                                                                | [01. &lt;= 19, 02. &lt;= 21, 03. &lt;= 24, 04. &gt;= 24]               | 4                   | Continue discrétisée |
| [`NBJFF10_MM_A`]                                                                  | [01. &lt;= 12, 02. &lt;= 15, 03. &lt;= 18, 04. &gt;= 18]               | 4                   | Continue discrétisée |
| [`NBJFF10_MSOM_A`]                                                                | [01. &lt;= 147, 02. &lt;= 178, 03. &lt;= 211, 04. &gt;= 211]           | 4                   | Continue discrétisée |
| [`NBJFF16_MMAX_A`]                                                                | [01. &lt;= 8, 02. &lt;= 10, 03. &lt;= 14, 04. &gt;= 14]                | 4                   | Continue discrétisée |
| [`NBJFF16_MM_A`]                                                                  | [01. &lt;= 3, 02. &lt;= 5, 03. &lt;= 7, 04. &gt;= 7]                   | 4                   | Continue discrétisée |
| [`NBJFF16_MSOM_A`]                                                                | [01. &lt;= 41, 02. &lt;= 59, 03. &lt;= 86, 04. &gt;= 86]               | 4                   | Continue discrétisée |
| [`NBJFF28_MMAX_A`]                                                                | [01. &lt;= 1, 02. &lt;= 2, 03. &lt;= 2, 04. &gt;= 2]                   | 4                   | Continue discrétisée |
| [`NBJFF28_MM_A`, `NBJFXI3S28_MM_A`, `NBJRR30_MM_A`, `NBJTX0_MM_A`]                      | [01. &lt;= 0, 02. &lt;= 1, 03. &lt;= 1, 04. &gt;= 1]                   | 4                   | Continue discrétisée |
| [`NBJFF28_MSOM_A`]                                                                | [01. &lt;= 5, 02. &lt;= 8, 03. &lt;= 12, 04. &gt;= 12]                 | 4                   | Continue discrétisée |
| [`NBJFXI3S10_MMAX_A`]                                                             | [01. &lt;= 18, 02. &lt;= 20, 03. &lt;= 22, 04. &gt;= 22]               | 4                   | Continue discrétisée |
| [`NBJFXI3S10_MM_A`]                                                               | [01. &lt;= 11, 02. &lt;= 13, 03. &lt;= 16, 04. &gt;= 16]               | 4                   | Continue discrétisée |
| [`NBJFXI3S10_MSOM_A`]                                                             | [01. &lt;= 135, 02. &lt;= 161, 03. &lt;= 189, 04. &gt;= 189]           | 4                   | Continue discrétisée |
| [`NBJFXI3S16_MMAX_A`, `NBJFXY10_MMAX_A`]                                            | [01. &lt;= 6, 02. &lt;= 9, 03. &lt;= 12, 04. &gt;= 12]                 | 4                   | Continue discrétisée |
| [`NBJFXI3S16_MM_A`]                                                               | [01. &lt;= 3, 02. &lt;= 4, 03. &lt;= 6, 04. &gt;= 6]                   | 4                   | Continue discrétisée |
| [`NBJFXI3S16_MSOM_A`]                                                             | [01. &lt;= 34, 02. &lt;= 48, 03. &lt;= 67, 04. &gt;= 67]               | 4                   | Continue discrétisée |
| [`NBJFXI3S28_MMAX_A`, `NBJFXY15_MM_A`, `NBJTXS32_MM_A`]                               | [01. &lt;= 1, 02. &lt;= 1, 03. &lt;= 2, 04. &gt;= 2]                   | 4                   | Continue discrétisée |
| [`NBJFXI3S28_MSOM_A`]                                                             | [01. &lt;= 4, 02. &lt;= 7, 03. &lt;= 10, 04. &gt;= 10]                 | 4                   | Continue discrétisée |
| [`NBJFXY10_MM_A`]                                                                 | [01. &lt;= 3, 02. &lt;= 4, 03. &lt;= 7, 04. &gt;= 7]                   | 4                   | Continue discrétisée |
| [`NBJFXY10_MSOM_A`]                                                               | [01. &lt;= 34, 02. &lt;= 52, 03. &lt;= 81, 04. &gt;= 81]               | 4                   | Continue discrétisée |
| [`NBJFXY15_MMAX_A`, `NBJTX35_MMAX_A`]                                               | [01. &lt;= 2, 02. &lt;= 3, 03. &lt;= 5, 04. &gt;= 5]                   | 4                   | Continue discrétisée |
| [`NBJFXY15_MSOM_A`]                                                               | [01. &lt;= 8, 02. &lt;= 15, 03. &lt;= 25, 04. &gt;= 25]                | 4                   | Continue discrétisée |
| [`NBJFXY8_MMAX_A`]                                                                | [01. &lt;= 10, 02. &lt;= 14, 03. &lt;= 17, 04. &gt;= 17]               | 4                   | Continue discrétisée |
| [`NBJFXY8_MSOM_A`]                                                                | [01. &lt;= 70, 02. &lt;= 99, 03. &lt;= 136, 04. &gt;= 136]             | 4                   | Continue discrétisée |
| [`NBJRR100_MMAX_A`, `NBJTN10_MM_A`, `NBJTNS25_MMAX_A`, `NBJTNS25_MSOM_A`, `NBJTX35_MM_A`] | [01. &lt;= 0, 02. &lt;= 0, 03. &lt;= 1, 04. &gt;= 1]                   | 4                   | Continue discrétisée |
| [`NBJRR100_MM_A`, `NBJRR50_MM_A`, `NBJTNS25_MM_A`]                                    | [01. &lt;= 0, 02. &lt;= 0, 03. &lt;= 0, 04. &gt;= 0]                   | 4                   | Continue discrétisée |
| [`NBJRR100_MSOM_A`, `NBJTNS20_MM_A`]                                                | [01. &lt;= 0, 02. &lt;= 1, 03. &lt;= 2, 04. &gt;= 2]                   | 4                   | Continue discrétisée |
| [`NBJRR10_MMAX_A`, `TN_VOR_MM_A`]                                                   | [01. &lt;= 5, 02. &lt;= 7, 03. &lt;= 8, 04. &gt;= 8]                   | 4                   | Continue discrétisée |
| [`NBJRR10_MM_A`]                                                                  | [01. &lt;= 2, 02. &lt;= 3, 03. &lt;= 4, 04. &gt;= 4]                   | 4                   | Continue discrétisée |
| [`NBJRR10_MSOM_A`]                                                                | [01. &lt;= 29, 02. &lt;= 36, 03. &lt;= 45, 04. &gt;= 45]               | 4                   | Continue discrétisée |
| [`NBJRR1_MMAX_A`]                                                                 | [01. &lt;= 13, 02. &lt;= 17, 03. &lt;= 19, 04. &gt;= 19]               | 4                   | Continue discrétisée |
| [`NBJRR1_MM_A`]                                                                   | [01. &lt;= 8, 02. &lt;= 10, 03. &lt;= 11, 04. &gt;= 11]                | 4                   | Continue discrétisée |
| [`NBJRR1_MSOM_A`]                                                                 | [01. &lt;= 90, 02. &lt;= 117, 03. &lt;= 132, 04. &gt;= 132]            | 4                   | Continue discrétisée |
| [`NBJRR30_MMAX_A`, `NBJTN5_MM_A`]                                                   | [01. &lt;= 1, 02. &lt;= 2, 03. &lt;= 3, 04. &gt;= 3]                   | 4                   | Continue discrétisée |
| [`NBJRR30_MSOM_A`]                                                                | [01. &lt;= 5, 02. &lt;= 8, 03. &lt;= 11, 04. &gt;= 11]                 | 4                   | Continue discrétisée |
| [`NBJRR50_MMAX_A`]                                                                | [01. &lt;= 1, 02. &lt;= 1, 03. &lt;= 1, 04. &gt;= 1]                   | 4                   | Continue discrétisée |
| [`NBJRR50_MSOM_A`]                                                                | [01. &lt;= 1, 02. &lt;= 3, 03. &lt;= 5, 04. &gt;= 5]                   | 4                   | Continue discrétisée |
| [`NBJRR5_MMAX_A`]                                                                 | [01. &lt;= 9, 02. &lt;= 10, 03. &lt;= 12, 04. &gt;= 12]                | 4                   | Continue discrétisée |
| [`NBJRR5_MM_A`]                                                                   | [01. &lt;= 4, 02. &lt;= 5, 03. &lt;= 6, 04. &gt;= 6]                   | 4                   | Continue discrétisée |
| [`NBJRR5_MSOM_A`]                                                                 | [01. &lt;= 46, 02. &lt;= 58, 03. &lt;= 71, 04. &gt;= 71]               | 4                   | Continue discrétisée |
| [`NBJTMS24_MMAX_A`]                                                               | [01. &lt;= 4, 02. &lt;= 7, 03. &lt;= 14, 04. &gt;= 14]                 | 4                   | Continue discrétisée |
| [`NBJTMS24_MM_A`]                                                                 | [01. &lt;= 1, 02. &lt;= 1, 03. &lt;= 3, 04. &gt;= 3]                   | 4                   | Continue discrétisée |
| [`NBJTMS24_MSOM_A`]                                                               | [01. &lt;= 8, 02. &lt;= 16, 03. &lt;= 30, 04. &gt;= 30]                | 4                   | Continue discrétisée |
| [`NBJTN10_MMAX_A`]                                                                | [01. &lt;= 1, 02. &lt;= 2, 03. &lt;= 5, 04. &gt;= 5]                   | 4                   | Continue discrétisée |
| [`NBJTN10_MSOM_A`]                                                                | [01. &lt;= 2, 02. &lt;= 5, 03. &lt;= 12, 04. &gt;= 12]                 | 4                   | Continue discrétisée |
| [`NBJTN5_MMAX_A`]                                                                 | [01. &lt;= 4, 02. &lt;= 6, 03. &lt;= 10, 04. &gt;= 10]                 | 4                   | Continue discrétisée |
| [`NBJTN5_MSOM_A`]                                                                 | [01. &lt;= 9, 02. &lt;= 19, 03. &lt;= 33, 04. &gt;= 33]                | 4                   | Continue discrétisée |
| [`NBJTNI10_MMAX_A`]                                                               | [01. &lt;= 15, 02. &lt;= 29, 03. &lt;= 30, 04. &gt;= 30]               | 4                   | Continue discrétisée |
| [`NBJTNI10_MM_A`]                                                                 | [01. &lt;= 18, 02. &lt;= 19, 03. &lt;= 22, 04. &gt;= 22]               | 4                   | Continue discrétisée |
| [`NBJTNI10_MSOM_A`]                                                               | [01. &lt;= 210, 02. &lt;= 233, 03. &lt;= 260, 04. &gt;= 260]           | 4                   | Continue discrétisée |
| [`NBJTNI15_MMAX_A`, `NBJTXI20_MMAX_A`, `NBJTXI27_MMAX_A`]                             | [01. &lt;= 15, 02. &lt;= 30, 03. &lt;= 30, 04. &gt;= 30]               | 4                   | Continue discrétisée |
| [`NBJTNI15_MM_A`]                                                                 | [01. &lt;= 23, 02. &lt;= 26, 03. &lt;= 27, 04. &gt;= 27]               | 4                   | Continue discrétisée |
| [`NBJTNI15_MSOM_A`]                                                               | [01. &lt;= 282, 02. &lt;= 309, 03. &lt;= 326, 04. &gt;= 326]           | 4                   | Continue discrétisée |
| [`NBJTNI20_MMAX_A`]                                                               | [01. &lt;= 8, 02. &lt;= 23, 03. &lt;= 30, 04. &gt;= 30]                | 4                   | Continue discrétisée |
| [`NBJTNI20_MM_A`]                                                                 | [01. &lt;= 14, 02. &lt;= 29, 03. &lt;= 29, 04. &gt;= 29]               | 4                   | Continue discrétisée |
| [`NBJTNI20_MSOM_A`]                                                               | [01. &lt;= 169, 02. &lt;= 344, 03. &lt;= 350, 04. &gt;= 350]           | 4                   | Continue discrétisée |
| [`NBJTNS20_MMAX_A`]                                                               | [01. &lt;= 1, 02. &lt;= 3, 03. &lt;= 7, 04. &gt;= 7]                   | 4                   | Continue discrétisée |
| [`NBJTNS20_MSOM_A`]                                                               | [01. &lt;= 3, 02. &lt;= 9, 03. &lt;= 20, 04. &gt;= 20]                 | 4                   | Continue discrétisée |
| [`NBJTX0_MMAX_A`]                                                                 | [01. &lt;= 2, 02. &lt;= 4, 03. &lt;= 7, 04. &gt;= 7]                   | 4                   | Continue discrétisée |
| [`NBJTX0_MSOM_A`]                                                                 | [01. &lt;= 4, 02. &lt;= 10, 03. &lt;= 18, 04. &gt;= 18]                | 4                   | Continue discrétisée |
| [`NBJTX25_MMAX_A`]                                                                | [01. &lt;= 12, 02. &lt;= 18, 03. &lt;= 23, 04. &gt;= 23]               | 4                   | Continue discrétisée |
| [`NBJTX25_MM_A`]                                                                  | [01. &lt;= 4, 02. &lt;= 5, 03. &lt;= 7, 04. &gt;= 7]                   | 4                   | Continue discrétisée |
| [`NBJTX25_MSOM_A`]                                                                | [01. &lt;= 44, 02. &lt;= 65, 03. &lt;= 88, 04. &gt;= 88]               | 4                   | Continue discrétisée |
| [`NBJTX30_MMAX_A`]                                                                | [01. &lt;= 6, 02. &lt;= 11, 03. &lt;= 17, 04. &gt;= 17]                | 4                   | Continue discrétisée |
| [`NBJTX30_MM_A`]                                                                  | [01. &lt;= 1, 02. &lt;= 2, 03. &lt;= 4, 04. &gt;= 4]                   | 4                   | Continue discrétisée |
| [`NBJTX30_MSOM_A`]                                                                | [01. &lt;= 15, 02. &lt;= 28, 03. &lt;= 44, 04. &gt;= 44]               | 4                   | Continue discrétisée |
| [`NBJTX35_MSOM_A`]                                                                | [01. &lt;= 2, 02. &lt;= 5, 03. &lt;= 9, 04. &gt;= 9]                   | 4                   | Continue discrétisée |
| [`NBJTXI20_MM_A`]                                                                 | [01. &lt;= 17, 02. &lt;= 19, 03. &lt;= 21, 04. &gt;= 21]               | 4                   | Continue discrétisée |
| [`NBJTXI20_MSOM_A`]                                                               | [01. &lt;= 207, 02. &lt;= 232, 03. &lt;= 256, 04. &gt;= 256]           | 4                   | Continue discrétisée |
| [`NBJTXI27_MM_A`]                                                                 | [01. &lt;= 24, 02. &lt;= 26, 03. &lt;= 27, 04. &gt;= 27]               | 4                   | Continue discrétisée |
| [`NBJTXI27_MSOM_A`]                                                               | [01. &lt;= 292, 02. &lt;= 311, 03. &lt;= 329, 04. &gt;= 329]           | 4                   | Continue discrétisée |
| [`NBJTXS32_MMAX_A`]                                                               | [01. &lt;= 4, 02. &lt;= 7, 03. &lt;= 11, 04. &gt;= 11]                 | 4                   | Continue discrétisée |
| [`NBJTXS32_MSOM_A`]                                                               | [01. &lt;= 9, 02. &lt;= 16, 03. &lt;= 27, 04. &gt;= 27]                | 4                   | Continue discrétisée |
| [`NB_CASERNES`]                                                                   | [01. &lt;= 1, 02. &lt;= 3, 03. &lt;= 14, 04. &gt;= 14]                 | 4                   | Continue discrétisée |
| [`PROPORTION_13`]                                                                 | [01. &lt;= 10, 02. &lt;= 20, 03. &lt;= 30, 04. &lt;= 40, 05. &lt;= 50] | 5                   | Continue discrétisée |
| [`RISK6`]                                                                         | [A, N, O]                                                              | 3                   | Nominale             |
| [`RRAB_VOR_MMAX_A`]                                                               | [01. &lt;= 41, 02. &lt;= 57, 03. &lt;= 82, 04. &gt;= 82]               | 4                   | Continue discrétisée |
| [`RRAB_VOR_MM_A`]                                                                 | [01. &lt;= 9, 02. &lt;= 19, 03. &lt;= 25, 04. &gt;= 25]                | 4                   | Continue discrétisée |
| [`RR_VOR_MMAX_A`]                                                                 | [01. &lt;= 137, 02. &lt;= 176, 03. &lt;= 232, 04. &gt;= 232]           | 4                   | Continue discrétisée |
| [`RR_VOR_MM_A`]                                                                   | [01. &lt;= 63, 02. &lt;= 79, 03. &lt;= 102, 04. &gt;= 102]             | 4                   | Continue discrétisée |
| [`TAMPLIAB_VOR_MM_A`, `TX_VOR_MM_A`]                                                | [01. &lt;= 7, 02. &lt;= 15, 03. &lt;= 18, 04. &gt;= 18]                | 4                   | Continue discrétisée |
| [`TAMPLIM_VOR_MMAX_A`]                                                            | [01. &lt;= 5, 02. &lt;= 12, 03. &lt;= 14, 04. &gt;= 14]                | 4                   | Continue discrétisée |
| [`TAMPLIM_VOR_MM_A`]                                                              | [01. &lt;= 4, 02. &lt;= 9, 03. &lt;= 10, 04. &gt;= 10]                 | 4                   | Continue discrétisée |
| [`TMMAX_VOR_MMAX_A`]                                                              | [01. &lt;= 12, 02. &lt;= 25, 03. &lt;= 26, 04. &gt;= 26]               | 4                   | Continue discrétisée |
| [`TMMAX_VOR_MM_A`]                                                                | [01. &lt;= 8, 02. &lt;= 17, 03. &lt;= 18, 04. &gt;= 18]                | 4                   | Continue discrétisée |
| [`TMMIN_VOR_MMAX_A`]                                                              | [01. &lt;= 7, 02. &lt;= 15, 03. &lt;= 17, 04. &gt;= 17]                | 4                   | Continue discrétisée |
| [`TMMIN_VOR_MM_A`]                                                                | [01. &lt;= 4, 02. &lt;= 6, 03. &lt;= 8, 04. &gt;= 8]                   | 4                   | Continue discrétisée |
| [`TMM_VOR_MMAX_A`]                                                                | [01. &lt;= 9, 02. &lt;= 18, 03. &lt;= 21, 04. &gt;= 21]                | 4                   | Continue discrétisée |
| [`TMM_VOR_MM_A`]                                                                  | [01. &lt;= 4, 02. &lt;= 9, 03. &lt;= 12, 04. &gt;= 12]                 | 4                   | Continue discrétisée |
| [`TM_VOR_MMAX_A`]                                                                 | [01. &lt;= 9, 02. &lt;= 19, 03. &lt;= 21, 04. &gt;= 21]                | 4                   | Continue discrétisée |
| [`TM_VOR_MM_A`]                                                                   | [01. &lt;= 5, 02. &lt;= 11, 03. &lt;= 12, 04. &gt;= 12]                | 4                   | Continue discrétisée |
| [`TNAB_VOR_MMAX_A`]                                                               | [01. &lt;= 7, 02. &lt;= 9, 03. &lt;= 11, 04. &gt;= 11]                 | 4                   | Continue discrétisée |
| [`TNAB_VOR_MM_A`]                                                                 | [01. &lt;= -1, 02. &lt;= 1, 03. &lt;= 2, 04. &gt;= 2]                  | 4                   | Continue discrétisée |
| [`TNMAX_VOR_MMAX_A`]                                                              | [01. &lt;= 9, 02. &lt;= 19, 03. &lt;= 20, 04. &gt;= 20]                | 4                   | Continue discrétisée |
| [`TNMAX_VOR_MM_A`]                                                                | [01. &lt;= 5, 02. &lt;= 11, 03. &lt;= 13, 04. &gt;= 13]                | 4                   | Continue discrétisée |
| [`TN_VOR_MMAX_A`]                                                                 | [01. &lt;= 6, 02. &lt;= 13, 03. &lt;= 15, 04. &gt;= 15]                | 4                   | Continue discrétisée |
| [`TXAB_VOR_MMAX_A`]                                                               | [01. &lt;= 16, 02. &lt;= 32, 03. &lt;= 35, 04. &gt;= 35]               | 4                   | Continue discrétisée |
| [`TXAB_VOR_MM_A`]                                                                 | [01. &lt;= 11, 02. &lt;= 22, 03. &lt;= 25, 04. &gt;= 25]               | 4                   | Continue discrétisée |
| [`TXMIN_VOR_MM_A`]                                                                | [01. &lt;= 6, 02. &lt;= 9, 03. &lt;= 11, 04. &gt;= 11]                 | 4                   | Continue discrétisée |
| [`TX_VOR_MMAX_A`]                                                                 | [01. &lt;= 12, 02. &lt;= 24, 03. &lt;= 27, 04. &gt;= 27]               | 4                   | Continue discrétisée |
| [`TYPBAT2`]                                                                       | [0, 1, 2]                                                              | 3                   | Ordinale             |
| [`ZONE_VENT`]                                                                     | [1.0, 2.0, 3.0]                                                        | 3                   | Ordinale             |



Ce résultat nous offre plusieurs leviers d’analyse et d’optimisation :

1. **Réduction de la redondance**
    - **Objectif** : Identifier les variables dupliquées en contenu, pour réduire la dimensionnalité ;
    - **Action** : Ne conserver qu’une seule variable par groupe de modalités identiques pour alléger les modèles ;
    - **Utilisation typique** : Préparation de données pour l'entraînement machine learning ou PCA.

2. **Analyse de cohérence ou de codage**
    - **Objectif** : Vérifier que des variables censées être différentes ne sont pas en réalité des duplicatas ou des transformations identiques ;
    - **Exemple** : Plusieurs indicateurs de risque (`CARACT1`, `EQUIPEMENT2`, `RISK10`, `RISK11`, `RISK12`, `RISK13`, `RISK9`) ayant les mêmes modalités `[N, O, R]` pourraient refléter un codage systématique ;
    - **Action** : Explorer l'origine des données ou les métadonnées pour trier les variables pertinentes.

3. Simplification des modèles prédictifs
    - **Objectif** : Réduire le nombre de variables d'entrée sans perte d'information ;
    - **Action** :
        - Éviter les colinéarités entre variables discrétisées identiques ;
        - Ne pas dupliquer des informations déjà présentes dans le modèle.
    - **Impact** : Meilleure généralisation, moins de surapprentissage ;

4. Regroupement thématique de variables
    - **Objectif** : Préparer une analyse factorielle ou thématique ;
    - **Action** : Créer des groupes de variables « équivalentes » en sens ou structure (ex. climat, socio-économie) ;
    - **Utilisation** : Interprétation, visualisation ou sélection par cluster.

5. Création de nouvelles features
    - **Objectif** : Utiliser le regroupement pour créer des variables dérivées ou indicateurs ;
    - **Exemple** : Une variable `IndicateurRisqueCommun` qui fusionne les `RISK*` à modalités `[N,O,R]` ;
    - **Impact** : Réduction du bruit, amélioration de la lisibilité.

6. Contrôle qualité des données
    - **Objectif** : Identifier les colonnes inutiles ou suspectes ;
    - **Action** :Supprimer ou réexaminer les variables ayant des modalités très limitées ou identiques à d'autres.    